This notebook is based on [this example](https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/semantic-search.ipynb#scrollTo=Fqo_hMRZiubM)

In [52]:
from pinecone import Pinecone, ServerlessSpec, PineconeApiException, Index
from dotenv import load_dotenv
import os
from typing import List, Dict, Any
import itertools

load_dotenv()


True

In [55]:
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

pc_index_name = "itax"
pc_index_dimension = 1024 # dimensionality of BAAI/bge-m3

# This method is convenient for testing but is not recommended for production:
# https://docs.pinecone.io/guides/data/target-an-index

def create_index_or_skip(index_name):
    try:
        if pc.has_index(index_name):
            print(f"Index {index_name} already exist, skip ...")
        else:
            pc.create_index(
                name=index_name,
                dimension=pc_index_dimension,
                metric="cosine",
                spec=ServerlessSpec(cloud="aws", region="us-east-1"),
            )
    except PineconeApiException as e:
        print(f"Failed to create index '{index_name}': {e}")

create_index_or_skip(pc_index_name)
pc_index = pc.Index(pc_index_name)


In [54]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

embedding_model_name = 'BAAI/bge-m3'

model = SentenceTransformer(embedding_model_name, device=device)
model


SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

# Prepare data and insert into Pinecone

In [56]:
# Example usage:
sample_records = [
    {
        "id": 1,
        "question": "ใครมีหน้าที่เสียภาษีเงินได้บุคคลธรรมดา?",
        "answer": "ใครมีหน้าที่เสียภาษีเงินได้บุคคลธรรมดา? ผู้มีหน้าที่เสียภาษีเงินได้บุคคลธรรมดา ได้แก่ ผู้ที่มีเงินได้เกิดขึ้นระหว่างปีที่ผ่านมาโดยมีสถานะ อย่างหนึ่งอย่างใด ดังนี้ 1) บุคคลธรรมดา 2) ห้างหุ้นส่วนสามัญหรือคณะบุคคลที่มิใช่นิติบุคคล 3) ผู้ถึงแก่ความตายระหว่างปีภาษี 4) กองมรดกที่ยังไม่ได้แบ่ง 5) วิสาหกิจชุมชน ตามกฎหมายว่าด้วยการส่งเสริมวิสาหกิจชุมชน เฉพาะที่เป็นห้างหุ้นส่วนสามัญ หรือคณะบุคคลที่มิใช่นิติบุคคล ปรับปรุงล่าสุด: 21-11-2020",
    },
    {
        "id": 2,
        "question": "เมื่อมีเงินได้เกิดขึ้นแล้ว ผู้มีหน้าที่เสียภาษีจะต้องทำอะไรบ้าง?",
        "answer": "เมื่อมีเงินได้เกิดขึ้นแล้ว ผู้มีหน้าที่เสียภาษีจะต้องทำอะไรบ้าง? ขอมีเลข และบัตรประจำตัวผู้เสียภาษี ภายใน 60 วัน นับแต่วันที่มีเงินได้เกิดขึ้น กรณีเป็นผู้มีเงินได้ ที่ไม่มีเลขประจำตัวประชาชน ได้แก่ เป็นคนต่างด้าว หรือกองมรดกที่ยังไม่ได้แบ่ง เว้นแต่ ผู้มีเงินได้ ที่มีเลขประจำตัวประชาชน สามารถใช้ เลขประจำตัวประชาชน แทนเลขประจำตัว ผู้เสียภาษีอากรได้ โดยไม่ต้องขอมีเลขประจำตัวผู้เสียภาษีอากรอีก ผู้มีเงินได้ที่มีภูมิลำเนา อยู่ใน กรุงเทพมหานคร อาจยื่นคำร้อง ณ สำนักงานสรรพากรพื้นที่ กรุงเทพมหานคร ทั้ง 30 แห่ง หรือ สำนักสรรพากรพื้นที่ สาขา(อำเภอ)ทุกแห่ง สำหรับใน ต่างจังหวัด ยื่นคำขอได้ที่สำนักงานสรรพากร พื้นที่ (จังหวัด) และสำนักงานสรรพากร พื้นที่สาขา (อำเภอ) ทุกแห่ง แล้วแต่กรณี ยื่นแบบแสดงรายการ ปกติปีละ 1 ครั้ง เงินได้ ของปีใด ก็ยื่นแบบฯ ภายใน วันที่ 31 มีนาคม ของปีถัดไป เว้นแต่ เงินได้ บางลักษณะ เช่น การให้เช่า ทรัพย์สิน เงินได้จาก วิชาชีพอิสระ เงินได้จาก การรับเหมา เงินได ้จากธุรกิจ การพาณิชย์ เป็นต้น จะต้อง ยื่นแบบฯ ตอนกลางปี สำหรับเงินได้ ที่เกิดขึ้นใน 6 เดือนแรก ภายใน เดือนกันยายน ของทุกปี ปรับปรุงล่าสุด: 21-11-2020",
    },
    {
        "id": 3,
        "question": "เงินได้อะไรบ้างที่ต้องเสียภาษี?",
        "answer": "เงินได้อะไรบ้างที่ต้องเสียภาษี? ตามกฎหมาย เงินได้ที่ต้องเสียภาษีเงินได้บุคคลธรรมดา เรียกว่า 'เงินได้พึงประเมิน' หมายถึง เงินได้ของบุคคลใดๆ หรือหน่วยภาษีใดข้างต้นที่เกิดขึ้นระหว่างวันที่ 1 มกราคม ถึง 31 ธันวาคม ของปีใดๆ หรือเงินได้ ที่เกิดขึ้นในปีภาษี ได้แก่ 1. เงิน 2. ทรัพย์สินซึ่งอาจคิดคำนวณได้เป็นเงิน ที่ได้รับจริง ที่ได้รับจริง(เกณฑ์เงินสด) 3. ประโยชน์ซึ่งอาจคิดคำนวณได้เป็นเงิน 4. เงินค่าภาษีอากรที่ผู้จ่ายเงินหรือผู้อื่นออกแทนให้ 5. เครดิตภาษีตามที่กฎหมายกำหนด ปรับปรุงล่าสุด: 21-11-2020",
    },
    {
        "id": 4,
        "question": "ผู้ที่มีเงินได้เกิดขึ้นในระหว่างปีภาษีมีหน้าที่ต้องยื่นแบบฯ เสียภาษีเงินได้บุคคลธรรมดาทุกกรณีหรือไม่?",
        "answer": "ผู้ที่มีเงินได้เกิดขึ้นระหว่างปีภาษีจะมีหน้าที่ต้องยื่นแบบฯ ก็ต่อเมื่อมีเงินได้ถึงเกณฑ์ขั้นต่ำตามที่กฎหมายกำหนด ไม่ว่าเมื่อคำนวณภาษีแล้วจะมีภาษีต้องชำระเพิ่มเติมหรือไม่ก็ตาม ดังนี้ เกณฑ์เงินได้พึงประเมินขั้นต่ำที่ผู้มีเงินได้ต้องยื่นแบบแสดงรายการภาษี (1) บุคคลธรรมดาและผู้ถึงแก่ความตาย มีเงินได้พึงประเมิน ดังนี้ ประเภทเงินได้ โสด สมรส เงินเดือนเพียงอย่างเดียว 120,000 220,000 เงินได้ประเภทอื่น 60,000 120,000 (2) ห้างหุ้นส่วนสามัญที่มิใช่นิติบุคคล หรือคณะบุคคลที่ไม่ใช่นิติบุคคล มีเงินได้พึงประเมินเกิน 60,000 บาท (3) กองมรดกที่ยังไม่ได้แบ่ง มีเงินได้พึงประเมินเกิน 60,000 บาท ปรับปรุงล่าสุด: 21-11-2020",
    },
    {
        "id": 5,
        "question": "เงินได้อันเป็นเหตุให้ต้องนำไปรวมคำนวณภาษีมาจากแหล่งใดบ้าง?",
        "answer": "แหล่งที่มาของเงินได้ ซึ่งแบ่งเป็นเงินได้จากแหล่งในประเทศและนอกประเทศ เงินได้จากแหล่งต่างๆ นี้จะต้องนำไปรวมคำนวณภาษีเงินได้บุคคลธรรมดาหรือไม่ ให้พิจารณา ดังนี้ 1. เงินได้เกิดจากแหล่งในประเทศ หมายถึง เงินได้ที่เกิดขึ้น หรือเป็นผลสืบเนื่องจากมี 1.1 หน้าที่งานที่ทำในประเทศไทย หรือ 1.2 กิจการที่ทำในประเทศไทย หรือ 1.3 กิจการของนายจ้างในประเทศไทย หรือ 1.4 ทรัพย์สินที่อยู่ในประเทศไทย (ดอกเบี้ย เงินปันผล ค่าเช่า ฯลฯ * เงื่อนไข ผู้มีเงินได้เกิดจากแหล่งในประเทศนี้มีหน้าที่ต้องเสียภาษีเงินได้ตามที่ประมวลรัษฏากร กำหนดไว้เสมอเว้นแต่จะมีข้อยกเว้นตามกฎหมาย ทั้งนี้ ไม่ว่าเงินได้พึงประเมินในปีภาษีที่ล่วงมาแล้วนั้น จะจ่ายในหรือนอกประเทศ และไม่ว่าผู้มีเงินได้นั้นจะเป็นผู้อยู่ในประเทศไทยหรือไม่ก็ตาม) 2. เงินได้เกิดจากแหล่งนอกประเทศไทย หมายถึง เงินได้ที่เกิดขึ้นหรือเป็นผลสืบเนื่องจากมี 2.1 หน้าที่งานที่ทำในต่างประเทศ หรือ 2.2 กิจการที่ทำในต่างประเทศ หรือ 2.3 ทรัพย์สินที่อยู่ในต่างประเทศ * เงื่อนไข ผู้มีเงินได้เกิดจากแหล่งนอกประเทศในปีภาษีที่ล่วงมาแล้วจะต้องเสียภาษีเงินได้ ในประเทศไทยก็ต่อเมื่อเข้าองค์ประกอบทั้ง 2 ประการ ดังต่อไปนี้ (1) ผู้มีเงินได้เป็น ผู้อยู่ในประเทศไทย ในปีภาษีนั้นชั่วระยะเวลาหนึ่งหรือหลายระยะเวลา รวมทั้งหมดถึง 180 วัน และ (2) ผู้มีเงินได้ นำเงินได้นั้นเข้ามาในประเทศไทยในปีภาษีนั้นด้วย ในการเสียภาษีเงินได้บุคคลธรรมดาบางกรณี ถ้าเกี่ยวข้องกับบุคคลของบางประเทศที่มี อนุสัญญาภาษีซ้อน* หรือความตกลงเพื่อป้องกันการเก็บภาษีซ้ำซ้อนกับประเทศไทยจำเป็นต้องพิจารณาถึงความ ตกลงหรืออนุสัญญาว่าด้วยการเว้นการเก็บภาษีซ้อนระหว่างประเทศไทยได้ทำความตกลงไว้ด้วย ปรับปรุงล่าสุด: 21-11-2020",
    },
    {
        "id": 6,
        "question": "ประเภทเงินรายได้ที่ต้องเสียภาษี",
        "answer": "เนื่องจากผู้มีเงินได้ประกอบอาชีพแตกต่างกัน มีความยากง่ายหรือต้นทุนที่แตกต่างกัน เพื่อความ เป็นธรรม ในกฎหมายจึงได้แบ่งลักษณะเงินได้(พึงประเมิน) ออกเป็นกลุ่มๆ ตามความเหมาะสมเพื่อกำหนด วิธีคำนวณภาษีให้เกิดความเป็นธรรมมากที่สุด ดังนี้ 1. เงินได้ประเภทที่ 1 ได้แก่ เงินได้เนื่องจากการจ้างแรงงาน ไม่ว่าจะเป็น - เงินเดือน ค่าจ้าง เบี้ยเลี้ยง โบนัส เบี้ยหวัด บำเหน็จ บำนาญ - เงินค่าเช่าบ้านที่ได้รับจากนายจ้าง - เงินที่คำนวณได้จากมูลค่าของการได้อยู่บ้าน ซึ่งนายจ้างให้อยู่โดยไม่เสียค่าเช่า - เงินที่นายจ้างจ่ายชำระหนี้ใด ๆ ซึ่งลูกจ้างมีหน้าที่ต้องชำระ - เงิน ทรัพย์สิน หรือประโยชน์ใด ๆ บรรดาที่ได้เนื่องจากการจ้างแรงงาน เช่น มูลค่าของการได้รับประทานอาหาร เป็นต้น 2. เงินได้ประเภทที่ 2 ได้แก่ เงินได้เนื่องจากหน้าที่หรือตำแหน่งงานที่ทำ หรือจากการรับทำงานให้ ไม่ว่าจะเป็น - ค่าธรรมเนียม ค่านายหน้า ค่าส่วนลด - เงินอุดหนุนในงานที่ทำ เบี้ยประชุม บำเหน็จ โบนัส - เงินค่าเช่าบ้านที่ได้รับเนื่องจากหน้าที่หรือตำแหน่งงานที่ทำ หรือจากการรับทำงานให้ - เงินที่คำนวณได้จากมูลค่าของการได้อยู่บ้าน ที่ผู้จ่ายเงินได้ให้อยู่โดยไม่เสียค่าเช่า - เงินที่ผู้จ่ายเงินได้จ่ายชำระหนี้ใด ๆ ซึ่งผู้มีเงินได้มีหน้าที่ต้องชำระ - เงิน ทรัพย์สิน หรือประโยชน์ใด ๆ บรรดาที่ได้เนื่องจากหน้าที่หรือตำแหน่งงานที่ทำหรือ จากการรับทำงานให้นั้น ไม่ว่าหน้าที่หรือตำแหน่งงาน หรืองานที่รับทำให้นั้นจะเป็นการประจำหรือชั่วคราว 3. เงินได้ประเภทที่ 3 ได้แก่ ค่าแห่งกู๊ดวิลล์ ค่าแห่งลิขสิทธิ์หรือสิทธิอย่างอื่น เงินปี หรือเงินได้ที่มีลักษณะ เป็นเงินรายปีอันได้มาจากพินัยกรรม นิติกรรมอย่างอื่น หรือคำพิพากษาของศาล 4. เงินได้ประเภทที่ 4 ได้แก่ ดอกเบี้ย เงินปันผล เงินส่วนแบ่งกำไร เงินลดทุน เงินเพิ่มทุน ผลประโยชน์ที่ได้จากการโอนหุ้น ฯลฯ เป็นต้น (ก) ดอกเบี้ยพันธบัตร ดอกเบี้ยเงินฝาก ดอกเบี้ยหุ้นกู้ ดอกเบี้ยตั๋วเงิน ดอกเบี้ยเงินกู้ยืม ไม่ว่าจะมี หลักประกันหรือไม่ ดอกเบี้ยเงินกู้ยืมที่อยู่ในบังคับต้องถูกหักภาษีไว้ ณ ที่จ่ายตามกฎหมายว่าด้วยภาษีเงินได้ปิโตรเลียมเฉพาะส่วนที่เหลือจากถูกหักภาษีไว้ ณ ที่จ่ายตามกฎหมายดังกล่าว หรือผลต่างระหว่างราคาไถ่ถอน กับราคาจำหน่ายตั๋วเงินหรือตราสารแสดงสิทธิในหนี้ที่บริษัท หรือห้างหุ้นส่วนนิติบุคคล หรือนิติบุคคลอื่น เป็นผู้ออกและจำหน่ายครั้งแรกในราคาต่ำกว่าราคาไถ่ถอน รวมทั้งเงินได้ที่มีลักษณะทำนองเดียวกันกับดอกเบี้ย ผลประโยชน์หรือค่าตอบแทนอื่น ๆ ที่ได้จากการให้กู้ยืมหรือจากสิทธิเรียกร้องในหนี้ทุกชนิดไม่ว่าจะมีหลักประกันหรือไม่ก็ตาม (ข) เงินปันผล เงินส่วนแบ่งของกำไร หรือประโยชน์อื่นใดที่ได้จากบริษัทหรือห้างหุ้นส่วนนิติบุคคล กองทุนรวม หรือสถาบันการเงินที่มีกฎหมายไทยให้จัดตั้งขึ้นโดยเฉพาะสำหรับให้กู้ยืมเงิน ฯลฯ (ค) เงินโบนัสที่จ่ายแก่ผู้ถือหุ้น หรือผู้เป็นหุ้นส่วนในบริษัทหรือห้างหุ้นส่วนนิติบุคคล (ง) เงินลดทุนของบริษัทหรือห้างหุ้นส่วนนิติบุคคลเฉพาะส่วนที่จ่ายไม่เกินกว่ากำไรและเงินที่กันไว้รวมกัน (จ) เงินเพิ่มทุนของบริษัทหรือห้างหุ้นส่วนนิติบุคคลซึ่งตั้งจากกำไรที่ได้มาหรือรับช่วงกันไว้รวมกัน (ฉ) ผลประโยชน์ที่ได้จากการที่บริษัทหรือห้างหุ้นส่วนนิติบุคคลควบเข้ากันหรือรับช่วงกันหรือ เลิกกัน ซึ่งตีราคาเป็นเงินได้เกินกว่าเงินทุน (ช) ผลประโยชน์ที่ได้จากการโอนการเป็นหุ้นส่วนหรือโอนหุ้น หุ้นกู้ พันธบัตร หรือตั๋วเงิน หรือ ตราสารแสดงสิทธิในหนี้ ที่บริษัทหรือห้างหุ้นส่วนนิติบุคคล หรือนิติบุคคลอื่นเป็นผู้ออก ทั้งนี้เฉพาะซึ่งตีราคา เป็นเงินได้เกินกว่าที่ลงทุน เงินได้ประเภทที่ 4 ในหลาย ๆ กรณี กฎหมายให้สิทธิที่จะเลือกเสียภาษีโดยวิธีหักภาษี ณ ที่จ่าย แทนการนำไปรวมคำนวณกับเงินได้อื่นตามหลักทั่วไป ซึ่งจะทำให้ผู้มีเงินได้ที่ต้องเสียภาษีตามบัญชีอัตราภาษี ในอัตราที่สูงกว่าอัตราภาษี หัก ณ ที่จ่าย สามารถประหยัดภาษีได้ 5. เงินได้ประเภทที่ 5 เงินได้จากการให้เช่าทรัพย์สิน เงินหรือประโยชน์อย่างอื่น ที่ได้เนื่องจาก - การให้เช่าทรัพย์สิน - การผิดสัญญาเช่าซื้อทรัพย์สิน - การผิดสัญญาซื้อขายเงินผ่อนซึ่งผู้ขายได้รับคืนทรัพย์สินที่ซื้อขายนั้นโดยไม่ต้องคืนเงินหรือประโยชน์ที่ได้รับไว้แล้ว 6. เงินได้ประเภทที่ 6 ได้แก่ เงินได้จากวิชาชีพอิสระ คือวิชากฎหมาย การประกอบโรคศิลป วิศวกรรม สถาปัตยกรรม การบัญชี ประณีตศิลปกรรม หรือวิชาชีพอื่นซึ่งจะได้มีพระราชกฤษฎีกากำหนดชนิดไว้ 7. เงินได้ประเภทที่ 7 ได้แก่ เงินได้จากการรับเหมาที่ผู้รับเหมาต้องลงทุนด้วยการจัดหาสัมภาระ ในส่วนสำคัญนอกจากเครื่องมือ 8. เงินได้ประเภทที่ 8 ได้แก่ เงินได้จากการธุรกิจ การพาณิชย์ การเกษตร การอุตสาหกรรม การขนส่ง การขายอสังหาริมทรัพย์ หรือการอื่นนอกจากที่ระบุไว้ในประเภทที่ 1 ถึงประเภทที่ 7 แล้ว ปรับปรุงล่าสุด: 21-11-2020",
    },
    {
        "id": 7,
        "question": "วิธีการคำนวณภาษีเงินได้บุคคลธรรมดาสิ้นปีจะต้องทำอย่างไร?",
        "answer": "โดยทั่วไปผู้มีเงินได้ต้องนำเงินได้พึงประเมินทุกประเภทของตน ตลอดปีภาษี (ไม่รวมเงินได้ที่กฎหมายยกเว้นภาษี หรือที่ไม่ต้องเสียภาษี) ไปคำนวณภาษีเงินได้บุคคลธรรมดาสิ้นปี เพื่อยื่นแบบแสดงรายการและชำระภาษีภายในเดือนมีนาคมของปีถัดจากปีที่มีเงินได้ การคำนวณภาษีให้ทำเป็น 3 ขั้น คือ ขั้นที่หนึ่ง คำนวณหาจำนวนภาษีตาม วิธีที่ 1 เสียก่อน การคำนวณภาษีตามวิธีที่ 1 เงินได้พึงประเมินทุกประเภทรวมกันตลอดปีภาษี xxxx (1) หัก ค่าใช้จ่ายตามที่กฎหมายกำหนด xxxx (2) (1)-(2) เหลือเงินได้หลังจากหักค่าใช้จ่าย xxxx (3) หัก ค่าลดหย่อนต่าง ๆ (ไม่รวมค่าลดหย่อนเงินบริจาค) ตามที่กฎหมายกำหนด xxxx (4) (3)-(4) เหลือเงินได้หลังจากหักค่าลดหย่อนต่าง ๆ xxxx (5) หัก ค่าลดหย่อนเงินบริจาค ไม่เกินจำนวนที่กฎหมายกำหนด xxxx (6) (5-6) เหลือเงินได้สุทธิ xxxx (7) นำเงินได้สุทธิตาม (7) ไปคำนวณภาษีตาม อัตราภาษีเงินได้บุคคลธรรมดา จำนวนภาษีตามการคำนวณภาษีวิธีที่ 1 xxxx (8) ขั้นที่สอง ให้พิจารณาว่าจะต้องคำนวณภาษีตาม วิธีที่ 2หรือไม่ ถ้าเข้าเงื่อนไขที่จะต้องคำนวณภาษีตามวิธีที่ 2 จึงคำนวณภาษีตามวิธีที่ 2 อีกวิธีหนึ่ง กรณีที่ต้องคำนวณภาษีตามวิธีที่ 2 ได้แก่ กรณีที่เงินได้พึงประเมินทุกประเภทในปีภาษี แต่ไม่รวม เงินได้พึงประเมินตามประเภทที่ 1 มีจำนวนรวมกันตั้งแต่ 120,000 บาทขึ้นไป การคำนวณภาษีตามวิธีที่ 2 นี้ ให้คำนวณในอัตราร้อยละ 0.5 ของยอดเงินได้พึงประเมิน (= เงินได้พึงประเมินทุกประเภทลบเงินได้พึงประเมินประเภทที่ 1 คูณด้วย 0.005) ดังกล่าวนั้น ขั้นที่สาม สรุป จำนวนภาษีที่ต้องเสียภาษี กำหนดให้ (10) คือ จำนวนภาษีที่คำนวณได้ตามวิธีที่ 2 การคำนวณภาษี จำนวนภาษีเงินได้สิ้นปีที่ต้องเสีย เทียบ (8) และ (10) จำนวนที่สูงกว่า xxxx (11) หัก ภาษีที่ถูกหัก ณ ที่จ่ายแล้ว xx ภาษีเงินได้ครึ่งปีที่ชำระไว้แล้ว xx ภาษีเงินได้ชำระล่วงหน้า xx เครดิตภาษีเงินปันผล xx xx (12) (11-12) เหลือ ภาษีเงินได้ที่ต้องเสีย (หรือที่เสียไว้เกินขอคืนได้) xx หมายเหตุ สำหรับเงินได้พึงประเมินที่ได้รับ ตั้งแต่วันที่ 1 มกราคม 2552 เป็นต้นไป หากคำนวณตามวิธีที่ 2 แล้วมีภาษีเงินได้ที่ต้องเสียจำนวนทั้งสิ้นไม่เกิน 5,000 บาท ผู้มีเงินได้ ได้รับการยกเว้นภาษีเงินได้ตามวิธีที่ 2 ( 10 ) แต่ยังคงมีหน้าที่เสียภาษีตามจำนวนที่คำนวณได้ตามวิธีที่ 1 ( 8 ) โดยนำมาสรุปจำนวนภาษีที่ต้องเสียขั้นที่สาม ปรับปรุงล่าสุด: 21-11-2020",
    },
    {
        "id": 8,
        "question": "เงินได้พึงประเมินแต่ละกรณีจะคำนวณหักค่าใช้จ่ายได้เท่าใด?",
        "answer": "ค่าใช้จ่ายเป็นองค์ประกอบหนึ่งในการคำนวณภาษี ถือเป็นสิทธิประโยชน์ทางภาษีอย่างหนึ่งที่กฎหมายกำหนดไว้ สำหรับหักเป็นต้นทุนในการทำงาน เพื่อให้ได้เงินได้หรือรายได้สุทธินั้นมาคิดภาษีตามบัญชีอัตราภาษี โดยมีอัตราการหักค่าใช้จ่ายมากหรือน้อยตามแต่ละประเภทของเงินได้ สรุปได้ดังนี้ ประเภทเงินได้ หักค่าใช้จ่าย 1. เงินเดือน ค่าจ้าง โบนัส เบี้ยเลี้ยง 50% ไม่เกิน 100,000 บาท หากมีเงินได้ประเภทที่ 1 และ 2 ให้นำเงินได้ทั้ง 2 ประเภท รวมกันแต่หักได้ไม่เกิน 100,000 บาท 2. เงินได้จากหน้าที่หรือตำแหน่งงานที่ทำ หรือจากการรับทำงานให้ ค่าธรรมเนียม ค่านายหน้า ฯลฯ 3. ค่าแห่งกู๊ดวิลล์ ค่าแห่งลิขสิทธิ์หรือสิทธิอย่างอื่น 50% ไม่เกิน 100,000 บาท หรือตามจริง 4. ดอกเบี้ย เงินปันผล ส่วนแบ่งกำไร ฯลฯ หักค่าใช้จ่ายไม่ได้ 5. รายได้จากการให้เช่าทรัพย์สิน การผิดสัญญาเช่าซื้อ การผิดสัญญาซื้อขายเงินผ่อน - บ้าน โรงเรือน สิ่งปลูกสร้าง แพ - ที่ดินที่ใช้ในการเกษตร - ที่ดินที่มิได้ใช้ในการเกษตร - ยานพาหนะ - ทรัพย์สินอื่น ตามจริงหรืออัตราเหมา 30% 20% 15% 30% 10% 6. วิชาชีพอิสระ - ประกอบโรคศิลปะ - กฎหมาย วิศวกรรม สถาปัตยกรรม บัญชี ประณีตศิลปกรรม ตามจริงหรืออัตราเหมา 60% 30% 7. รับเหมาก่อสร้าง ตามจริงหรืออัตราเหมา 60% 8. รายได้อื่น นอกเหนือจาก 1-7 * ตามจริงหรืออัตราเหมา 40% และ 60% * ตามพระราชกฤษฎีกา (ฉบับที่ 629) พ.ศ.2560 ปรับปรุงล่าสุด: 21-11-2020",
    },
    {
        "id": 9,
        "question": "ผู้มีเงินได้มีหน้าที่ต้องยื่นแบบแสดงรายการภาษีเงินได้บุคคลธรรมดาอย่างไร และเมื่อใด?",
        "answer": "บุคคลธรรมดา ความรู้เรื่องภาษี ภาษีเงินได้บุคคลธรรมดา กำหนดเวลายื่นแบบ กำหนดเวลายื่นแบบ ภาษีเงินได้บุคคลธรรมดา ใครมีหน้าที่เสียภาษีเงินได้บุคคลธรรมดา เมื่อมีเงินได้เกิดขึ้นแล้วผู้มีหน้าที่เสียภาษีจะต้องทำอะไรบ้าง เงินได้อะไรบ้างที่ต้องเสียภาษี ผู้ที่มีเงินได้เกิดขึ้นในระหว่างปีภาษีมีหน้าที่ต้องยื่นแบบฯ เสียภาษีเงินได้บุคคลธรรมดาทุกกรณีหรือไม่ เงินได้อันเป็นเหตุให้ต้องนำไปรวมคำนวณภาษีมาจากแหล่งใดบ้าง ประเภทเงินได้ที่ต้องเสียภาษี ประเภทเงินได้ที่ได้รับยกเว้นภาษี วิธีการคำนวณภาษีเงินได้บุคคลธรรมดาสิ้นปี การหักค่าใช้จ่าย การหักลดหย่อน กำหนดเวลายื่นแบบ สถานที่ยื่นแบบแสดงรายการ วิธีการชำระภาษี กรณีไม่ชำระภาษีตามกำหนดเวลา อัตราภาษี การบริจาคเงินภาษีให้แก่พรรคการเมือง การขอผ่อนชำระภาษี การขอคืนเงินภาษีอากร ข้อคิดสะกิดใจการยื่นแบบ ภ.ง.ด.90,91 ผู้มีเงินได้มีหน้าที่ต้องยื่นแบบแสดงรายการภาษีเงินได้บุคคลธรรมดาอย่างไร และเมื่อใด? แบบแสดงรายการที่ใช้มีดังต่อไปนี้ ชื่อแบบ ใช้ยื่นกรณี กำหนดเวลายื่น ภ.ง.ด. 90 มีเงินได้พึงประเมินทุกประเภท มกราคม - มีนาคม ของปีภาษีถัดไป ภ.ง.ด. 91 มีเฉพาะเงินได้พึงประเมินประเภทที่ 1ม.40(1) ประเภทเดียว มกราคม - มีนาคม ของปีภาษีถัดไป ภ.ง.ด. 93 มีเงินได้ขอชำระภาษีล่วงหน้า ก่อนถึงกำหนดเวลาการยื่นแบบตามปกติ ภ.ง.ด. 94 ยื่นครึ่งปีสำหรับผู้มีเงินได้พึงประเมินเฉพาะประเภทที่ 5,6,7 และ 8 กรกฎาคม - กันยายน ของปีภาษีนั้น ภ.ง.ด. 95 คนต่างด้าวผู้มีเงินได้จากการาจ้างแรงงานจากสำนักงานปฏิบัติการภูมิภาค มกราคม - มีนาคม ของปีภาษีถัดไป การยื่นแบบแสดงรายการภาษีเงินได้บุคคลธรรมดา มี 2 ระยะ คือ 1. 'ภาษีเงินได้บุคคลธรรมดาครึ่งปี' เป็นการยื่นแบบแสดงรายการเงินได้เฉพาะเงินได้พึงประเมินประเภทที่ 5,6,7 หรือ 8 ที่ได้รับตั้งแต่เดือนมกราคมถึงเดือนมิถุนายนไม่ว่าจะมีเงินได้ประเภทอื่นรวมอยู่ด้วยหรือไม่ก็ตาม โดยยื่นภายในเดือนกันยายนของปีภาษีนั้น และภาษีที่เสียนี้นำไปเป็นเครดิตหักออกจากภาษีสิ้นปีได 2. 'ภาษีเงินได้บุคคลธรรมดาสิ้นปี' เป็นการยื่นแบบแสดงรายการเงินได้พึงประเมินที่ได้รับแล้ว ในระหว่างปีภาษี โดยยื่นภายในเดือนมีนาคมของปีถัดไป การยื่นแบบ ภ.ง.ด. 90 91 ผ่านอินเทอร์เน็ต ขั้นตอนการยื่นแบบฯ 1. เข้า web site ของกรมสรรพากรที่ www.rd.go.th 2. เลือกรายการ e-FILING 3. เข้าใช้บริการครั้งแรก ให้เลือกรายการสมัครใช้บริการ 4. หลังจากนั้น เข้าระบบตามขั้นตอน ดังนี้ 5. ป้อนรายการข้อมูล ได้แก่ รายการเงินได้ ค่าลดหย่อน เงินได้ที่ได้รับยกเว้นภาษีหัก ณ ที่จ่าย ฯลฯ แล้ว คลิก 'ตกลง' เพื่อยืนยันการยื่นแบบฯ 6. เมื่อได้ตรวจสอบรายการข้อมูลที่บันทึกและสั่งให้ระบบ 'คำนวณภาษีแล้ว' 6.1 กรณีไม่มีภาษีต้องชำระ (1) โปรแกรมจะแจ้งผลการรับแบบและหมายเลขอ้างอิง เพื่อใช้เป็นหลักฐานในการยื่นแบบฯ (2) กรมสรรพากรจะส่งใบเสร็จรับเงินทันที 6.2 กรณีมีภาษีต้องชำระ (1) หากเลือกวิธีชำระภาษีผ่าน e-payment ระบุธนาคารที่ท่านใช้บริการอยู่ และดำเนินการตามขั้นตอนของ ธนาคารนั้น (2) หากเลือกวิธีชำระภาษีผ่านระบบอิเล็กทรอนิกส์ของธนาคาร ข้อมูลรายละเอียดเกี่ยวกับการชำระภาษีจะถูกส่งจากกรมสรรพากรไปยังธนาคารโดยตรง เมื่อผู้เสียภาษียืนยันการทำรายการของระบบชำระเงินของธนาคารแล้ว ธนาคารจะแจ้งผลการทำรายการ เช่น ผลการโอนเงิน การยกเลิกการทำรายการ เป็นต้น ส่งให้กรมสรรพากรแบบทันทีทันใด (Real Time) (3) หากเลือกวิธีชำระภาษี ณ เคาน์เตอร์ ไปรษณีย์อัตโนมัติ (Pay at Post) ระบบจะแจ้งรายการข้อมูลเช่นเดียวกับ (2) เพื่อใช้เป็นข้อมูลนำไปชำระภาษี ณ ที่ทำการไปรษณีย์ทุกแห่ง (ยกเว้น ปณ. โสกเชือก จ. ร้อยเอ็ด และ ปณ. ชุมแสงสงคราม จ.พิษณุโลก ขั้นตอนการชำระภาษี เมื่อได้ทำรายการยื่นแบบ ภ.ง.ด.90 ภ.ง.ด.91 ผ่านอินเทอร์เน็ตเรียบร้อยแล้ว และเป็นกรณีที่มีภาษีต้องชำระ ผู้ใช้บริการจะต้องชำระภาษีทั้งจำนวนในวันใดก็ได้ภายในกำหนดเวลายื่นแบบ โดยมีทางเลือกในการชำระภาษีวิธีใดวิธีหนึ่ง ดังนี้ 1. การชำระภาษีผ่านบริการอิเล็กทรอนิกส์ของธนาคารพาณิชย์ 1.1 การชำระภาษีผ่าน e-payment เป็นระบบการชำระเงินพร้อมกับการยื่นรายการข้อมูลตามแบบ (1) ผู้ใช้บริการต้องทำความตกลงกับธนาคารไว้แล้ว (2) เลือกธนาคารที่ต้องการสั่งโอนเงินจากบัญชีเงินฝากธนาคารเพื่อชำระภาษี (3) ทำรายการโอนเงินตามขั้นตอนของธนาคาร (4) เมื่อทำรายการโดยครบถ้วนแล้วโปรแกรมจะแจ้ง หมายเลขอ้างอิง การยื่นแบบฯ และชำระภาษีให้ (5) กรมสรรพากรจะส่งใบเสร็จรับเงินทางไปรษณีย์ลงทะเบียนตามที่อยู่ที่แสดงในแบบให้โดยเร็ว 1.2 การชำระวิธีอื่น (1) เลือกบริการชำระภาษี (2) ป้อนข้อมูลหมายเลขประจำตัวผู้เสียภาษี รหัสควบคุม และจำนวนภาษีที่ต้องชำระที่ได้จากโปรแกรมการยื่นแบบฯ ผ่านอินเทอร์เน็ต (3) หากข้อมูลตามข้อ (2) ถูกต้องธนาคารจะโอนเงินจากบัญชีเงินฝากของท่านเข้าบัญชีกรมสรรพากรเพื่อชำระภาษี (4) กรมสรรพากร จะส่งใบเสร็จรับเงินทางไปรษณีย์ลงทะเบียนตามที่อยู่ที่ได้ลงทะเบียนไว้ในแบบฯให้โดยเร็ว หมายเหตุ กรณีชำระผ่านเครื่อง ATM ต้องเป็นเครื่อง ATM ของธนาคารผู้ออกบัตรเท่านั้น 2. การชำระเงินทางไปรษณีย์ Pay at Post ให้นำรายการข้อมูลที่ได้รับจากระบบได้แก่ เลขประจำตัวผู้เสียภาษีอากร รหัสควบคุม จำนวนเงินภาษี ไปชำระเงินภาษีอากรได้ที่ทำการไปรษณีย์ทั่วประเทศ (ยกเว้น ปณ. โสกเชือก จ.ร้อยเอ็ด และ ปณ.ชุมแสงสงคราม จ. พิษณุโลก) กรมสรรพากรจะส่งใบเสร็จรับเงินทางไปรษณีย์ลงทะเบียนตามที่อยู่ที่ได้ลงทะเบียนไว้ให้โดยเร็ว ปรับปรุงล่าสุด: 13-02-2023",
    },
    {
        "id": 10,
        "question": "การชำระภาษีด้วยวิธีใดบ้าง ?",
        "answer": "การชำระภาษีเงินได้บุคคลธรรมดา นอกจากผู้เสียภาษี จะชำระภาษีโดยการ ถูกหักภาษี ณ ที่จ่ายแล้ว ถ้าในการยื่นแบบ แสดงรายการ และคำนวณภาษี ตามแบบแสดงรายการ ที่ยื่นนั้น มีภาษีที่ต้องชำระ หรือต้องชำระเพิ่มเติมอีก ก็ให้ชำระ หรือชำระเพิ่มเติม ต่อเจ้าหน้าที่สรรพากร พร้อมกับ การยื่นแบบนั้น โดยเจ้าหน้าที่ผู้รับชำระ จะออกหลักฐานใบเสร็จ แสดงการรับเงินภาษี และถือเป็นหลักฐาน แสดงการยื่นแบบฯ ให้กับ ผู้ยื่นแบบแสดงรายการ ทุกราย การชำระภาษีเลือกวิธีการชำระได้ดังนี้ 1. ชำระด้วยเงินสด 2. ชำระด้วยบัตรอิเล็กทรอนิกส์ (โดยผู้ถือบัตรอิเล็กทรอนิกส์เป็นผู้ชำระค่าธรรมเนียม) สามารถชำระภาษีได้ ณ สำนักงานสรรพากรพื้นที่สาขา ดังนี้ (1) สำนักงานสรรพากรพื้นที่สาขาในเขตกรุงเทพมหานคร (2) สำนักงานสรรพากรพื้นที่สาขาในเขตจังหวัดชลบุรี ได้แก่ สำนักงานสรรพากรพื้นที่สาขาเมืองชลบุรี 1 สำนักงานสรรพากรพื้นที่สาขาพานทอง สำนักงานสรรพากรพื้นที่สาขาศรีราชา 1 สำนักงานสรรพากรพื้นที่สาขาศรีราชา 2 สำนักงานสรรพากรพื้นที่สาขาบางละมุง 1 และสำนักงานสรรพากรพื้นที่สาขาบางละมุง 2 (3) สำนักงานสรรพากรพื้นที่สาขาในเขตจังหวัดสมุทรปราการ ได้แก่ สำนักงานสรรพากรพื้นที่สาขาเมืองสมุทรปราการ 1 สำนักงานสรรพากรพื้นที่สาขาเมืองสมุทรปราการ 2 สำนักงานสรรพากรพื้นที่สาขา พระประแดง 1 สำนักงานสรรพากรพื้นที่สาขาพระประแดง 2 และสำนักงานสรรพากรพื้นที่สาขาพระสมุทรเจดีย์ (4) สำนักงานสรรพากรพื้นที่สาขาในเขตจังหวัดฉะเชิงเทราได้แก่ สำนักงานสรรพากรพื้นที่สาขา เมืองฉะเชิงเทรา และสำนักงานสรรพากรพื้นที่สาขาบางปะกง (5) สำนักงานสรรพากรพื้นที่สาขาในเขตจังหวัดระยอง ได้แก่ สำนักงานสรรพากรพื้นที่สาขาเมืองระยอง 2 และสำนักงานสรรพากรพื้นที่สาขาปลวกแดง (6) สำนักงานสรรพากรพื้นที่สาขาในเขตจังหวัดนนทบุรี ได้แก่ สำนักงานสรรพากรพื้นที่สาขาเมืองนนทบุรี 1 สำนักงานสรรพากรพื้นที่สาขาเมืองนนทบุรี 2 สำนักงานสรรพากรพื้นที่สาขาปากเกร็ด 1 สำนักงาน สรรพากรพื้นที่สาขาปากเกร็ด 2 สำนักงานสรรพากรพื้นที่สาขาบางกรวย และสำนักงานสรรพากรพื้นที่สาขา บางบัวทอง (7) สำนักงานสรรพากรพื้นที่สาขาในเขตจังหวัดปทุมธานี ได้แก่ สำนักงานสรรพากรพื้นที่สาขาเมืองปทุมธานี สำนักงานสรรพากรพื้นที่สาขาคลองหลวง 1 สำนักงานสรรพากรพื้นที่สาขาคลองหลวง 2 สำนักงาน สรรพากรพื้นที่สาขาธัญบุรี และสำนักงานสรรพากรพื้นที่สาขาลำลูกกา (8) สำนักงานสรรพากรพื้นที่สาขาในเขตจังหวัดพระนครศรีอยุธยา ได้แก่ สำนักงานสรรพากรพื้นที่สาขาบางปะอิน สำนักงานสรรพากรพื้นที่สาขาวังน้อย และสำนักงานสรรพากรพื้นที่สาขาอุทัย (9) สำนักงานสรรพากรพื้นที่สาขาในเขตจังหวัดสระบุรี ได้แก่ สำนักงานสรรพากรพื้นที่สาขาแก่งคอย และสำนักงานสรรพากรพื้นที่สาขาหนองแค 2.1 บัตรเครดิต TAX SMART CARD และบัตรเดบิต ของ บมจ. บัตรกรุงไทย (KTC) ธนาคารกรุงไทย (KTB) สามารถชำระภาษีได้ ณ สำนักงานสรรพากรพื้นที่สาขาตาม (1) – (9) 2.2 บัตรเครดิต และ TAX SMART CARD ของธนาคาร กสิกรไทย (KBANK) และธนาคารกรุงเทพ (BBL) สามารถชำระภาษีได้ ณ สำนักงานสรรพากรพื้นที่สาขาตาม (1) – (5) 2.3 บัตรเครดิต ของธนาคารไทยพาณิชย์ (SCB) สามารถชำระภาษีได้ ณ สำนักงานสรรพากรพื้นที่สาขา ตาม (1) - (5) 3. ชำระด้วยเช็คหรือดราฟต์ 3.1 เช็คที่ชำระต้องเป็นเช็ค 4 ประเภท ได้แก่ (1) เช็คธนาคารแห่งประเทศไทย (เช็คประเภท ก.) (2) เช็คที่มีธนาคารค้ำประกัน (เช็คประเภท ข.) (3) เช็คที่ธนาคารเซ็นสั่งจ่าย (เช็คประเภท ค.) (4) เช็คที่ผู้มีหน้าที่ชำระเงินภาษีอากรเป็นผู้เซ็นสั่งจ่าย และใช้ชำระโดยตรง (เช็คประเภท ง.) การใช้เช็คประเภท ง. ให้ปฏิบัติดังนี้ ( ก ) กรณีใช้ชำระภาษีในกรุงเทพมหานคร และในจังหวัดนนทบุรี จังหวัดปทุมธานี จังหวัดสมุทรปราการ จังหวัดสมุทรสาคร (เฉพาะอำเภอเมืองสมุทรสาคร และอำเภอกระทุ่มแบน) จังหวัดนครปฐม (เฉพาะอำเภอสามพราน) และจังหวัดพระนครศรีอยุธยา (เฉพาะอำเภอบางปะอิน) ต้องเป็นเช็คของธนาคาร หรือสาขาธนาคาร ซึ่งตั้งอยู่ใน ท้องที่ใดท้องที่หนึ่งข้างต้นเท่านั้น ( ข ) กรณีใช้ชำระภาษีในจังหวัด (อำเภอ) อื่นนอกจาก (ก) ให้ใช้ได้เฉพาะกรณีที่เปิดบัญชีเงินฝากไว้กับธนาคารหรือ สาขาธนาคารซึ่งตั้งอยู่ในจังหวัดท้องที่อันเป็นภูมิลำเนาเท่านั้น 3.2 การสั่งจ่ายเช็คหรือดราฟต์ ให้ขีดคร่อม และสั่งจ่ายดังนี้ (1) ในกรุงเทพมหานคร กรณียื่นแบบ ภ.ง.ด.90 หรือ ภ.ง.ด.91 และชำระภาษีที่สำนักงานสรรพากรพื้นที่สาขา เช็คทุกประเภทให้สั่งจ่ายแก่ “กรมสรรพากร” และขีดฆ่าคำว่า “ผู้ถือ” ออก (2) ในต่างจังหวัด กรณียื่นแบบ ภ.ง.ด.90 หรือ ภ.ง.ด.91 และชำระภาษีที่สำนักงานสรรพากร พื้นที่สาขาด้วยเช็คประเภท ง. ให้สั่งจ่ายแก่ “กรมสรรพากร” ถ้าเป็นเช็คประเภท ก. ข. ค. ให้ติดต่อสำนักงานสรรพากรพื้นที่สาขา หลักเกณฑ์การใช้เช็คหรือดราฟต์ (1) กรณียื่นที่ สำนักงานสรรพากรพื้นที่สาขา เช็คทุกประเภทหรือดราฟต์ ต้องลง วันที่ในเช็ค ในวันที่ยื่นแบบฯ หรือ ก่อนวันที่ที่ยื่นแบบฯ ไม่เกิน 15 วัน สำหรับเช็คประเภท ก. ข. ค. หรือดราฟต์ หรือไม่เกิน 7 วัน สำหรับเช็คประเภท ง. (2) กรณียื่นที่ธนาคาร/ที่ทำการไปรษณีย์ เช็คทุกประเภท หรือดราฟต์ที่ชำระภาษี ต้องลงวันที่ในเช็คในวันที่ยื่นแบบฯ หรือก่อนวันที่ยื่นแบบฯ ไม่เกิน 7 วัน (3) ห้ามใช้เช็คลงวันที่ล่วงหน้า (4) ห้ามใช้เช็คหรือดราฟต์โอนสลักหลัง (5) ห้ามใช้เช็คหรือดราฟต์ที่มีจำนวนเงินสูงกว่าจำนวนภาษีอากรที่ต้องชำระ (6) การชำระภาษี จะถือว่าสมบูรณ์ ต่อเมื่อ กรมสรรพากร ได้รับเงิน ตามเช็ค หรือ ดราฟต์ครบถ้วนแล้ว 4.ชำระด้วยธนาณัติ 1. ผู้มีเงินได้ที่ยื่นแบบฯ ภ.ง.ด.90 หรือ ภ.ง.ด.91 ต้องมีภูมิลำเนาอยู่ในกรุงเทพมหานคร 2. ส่งธนาณัติเท่ากับจำนวนเงินภาษีที่ต้องชำระ ไปพร้อมกับการยื่นแบบ ภ.ง.ด.90 หรือ ภ.ง.ด.91 (ห้ามหักค่าธรรมเนียม ในการส่งธนาณัติ) โดยสั่งจ่าย “ผู้อำนวยการกองบริหารการคลังและรายได้ กรมสรรพากร” ปณ.กระทรวงการคลัง ปรับปรุงล่าสุด: 21-11-2020",
    },
    {
        "id": 11,
        "question": "ถ้าไม่ชำระในกำหนดเวลาหรือชำระไม่ถูกต้องจะมีความรับผิดอย่างไรบ้าง?",
        "answer": "เบี้ยปรับและเงินเพิ่ม ถือเป็นบทลงโทษเกี่ยวกับภาษีอากรอย่างหนึ่ง และอาจมีโทษทางอาญาด้วย แล้วแต่ความผิดว่าเป็นเรื่องร้ายแรงขั้นไหน โดยหากบุคคลใดยื่นแบบฯ ภายในกำหนดแต่ชำระภาษีไม่ครบถ้วน หรือยื่นแบบฯ ล่าช้า ละเลย หรือหลีกเลี่ยงการยื่นแบบฯ จะต้องเสียเงินเพิ่มและเบี้ยปรับตามกฎหมายกำหนด และหากฝ่าฝืนไม่ยอมชำระ ก็ต้องรับโทษทางอาญาด้วย ซึ่งมีบทลงโทษ ดังนี้ กรณีไม่ชำระภาษีภายในกำหนดเวลา จะต้องเสียเงินเพิ่มอีกร้อยละ 1.5 ต่อเดือน (เศษของเดือนให้นับเป็น 1 เดือน) ของเงินภาษีที่ต้องชำระนับแต่วันพ้นกำหนดเวลาการยื่นรายการจนถึงวันชำระภาษี กรณีเจ้าพนักงานตรวจสอบออกหมายเรียก และปรากฏว่ามิได้ยื่นแบบแสดงรายการไว้หรือยื่นแบบแสดงรายการไว้แต่ชำระภาษีขาดหรือต่ำไป นอกจากจะต้องรับผิดชำระเงินเพิ่มแล้ว ยังจะต้องรับผิดเสียเบี้ยปรับอีก 1 เท่าหรือ 2 เท่าของภาษีที่ต้องชำระแล้วแต่กรณี เงินเบี้ยปรับดังกล่าวอาจลดหรืองดได้ตามระเบียบที่อธิบดีกำหนดโดยอนุมัติรัฐมนตรี กรณีไม่ยื่นแบบแสดงรายการ ภ.ง.ด.90, 91 หรือ 94 ภายในกำหนดเวลา ต้องระวางโทษปรับทางอาญาไม่เกิน 2,000 บาท กรณีจงใจ แจ้งข้อความเท็จ หรือแสดงหลักฐานเท็จหรือฉ้อโกง เพื่อหลีกเลี่ยงหรือพยายามหลีกเลี่ยงการเสียภาษีอากร มีโทษจำคุกตั้งแต่ 3 เดือนถึง 7 ปี และปรับตั้งแต่ 2,000 บาท ถึง 200,000 บาท กรณีเจตนาละเลยไม่ยื่นแบบแสดงรายการเพื่อหลีกเลี่ยงการเสียภาษีอากร มีโทษปรับไม่เกิน 200,000 บาท หรือจำคุกไม่เกิน 1 ปี หรือทั้งจำทั้งปรับ ปรับปรุงล่าสุด: 21-11-2020",
    },
    {
        "id": 12,
        "question": "การบริจาคเงินภาษีให้แก่พรรคการเมือง",
        "answer": "การบริจาคเงินภาษีให้แก่พรรคการเมือง ตั้งแต่ปีภาษี 2551 เป็นต้นไป บุคคลธรรมดา ที่มีสัญชาติไทย (ไม่รวมถึง ห้างหุ้นส่วนสามัญ หรือ คณะบุคคล ที่มิใช่ นิติบุคคล และกองมรดก ที่ยังไม่ได้แบ่ง) สามารถบริจาค เงินภาษี ให้แก่พรรคการเมืองได้ ตามมาตรา 58 แห่งพระราชบัญญัต ประกอบรัฐธรรมนูญ ว่าด้วยพรรคการเมือง พ.ศ.2550 โดยแสดง เจตนาบริจาค เงินภาษี ให้แก่พรรคการเมือง พร้อมการยื่นแบบ แสดงรายการภาษี ประจำปี โดยมีหลักเกณฑ์ และวิธีการ ตามประกาศอธิบดีฯ ภาษีเงินได้ ( ฉบับที่ 176 ) ดังนี้ 1. ผู้มีเงินได้ เมื่อคำนวณภาษี ตามแบบ ภ.ง.ด.90 หรือ ภ.ง.ด.91 แล้ว มีเงินภาษี ที่ต้องชำระ ตั้งแต่ 100 บาท ขึ้นไป 2. ผู้มีเงินได้ ที่มีสิทธิบริจาค ต้องแสดง เจตนาไว้ใน ช่องที่กำหนดไว้ ในแบบ ภ.ง.ด.90 หรือ ภ.ง.ด.91 โดยต้อง ระบุให้ชัดเจนว่า ประสงค์จะบริจาค หรือ ไม่บริจาค และระบุ รหัสพรรคการเมือง ที่ต้องการบริจาค หากไม่ระบ ุความประสงค์ หรือไม่ระบุ รหัสพรรคการเมือง ถือว่าไม่ได้แสดง เจตนาบริจาค ตามหลักเกณฑ์ ดังนี้ ( 1 ) ระบุรหัสพรรคการเมือง ที่ต้องการบริจาค ได้เพียง 1 พรรคการเมือง หากแสดงเจตนา เกินกว่า 1 พรรคการเมือง ถือว่า ไม่ประสงค์ จะบริจาคให้พรรคการเมืองใด ( 2 ) เมื่อแสดงเจตนาบริจาค ให้แก่พรรคการเมืองใดแล้ว ห้ามปลี่ยนแปลง ( 3 ) พรรคการเมือง ที่ผู้บริจาค จะแสดงเจตนา บริจาคเงินภาษี ให้ได้ในปีภาษีใด จะต้องเป็น พรรคการเมือง ที่มีชื่ออยู่ใน ทะเบียนพรรคการเมือง ในปีภาษีนั้น หากพรรคการเมือง ที่แสดงความประสงค์ บริจาคเงินภาษี ให้สิ้นสภาพ เลิก หรือยุบ ตามกฎหมาย ในปีภาษีใด ให้ถือเสมือนว่า ไม่มีพรรคการเมืองนั้น ที่จะได้รับการแสดงเจตนา บริจาคเงินภาษีในปีภาษีนั้น สามารถค้นหา รหัสพรรคการเมือง ได้จากเว็บไซต์สำนักงานคณะกรรมการเลือกตั้ง www.ect.go.th หรือจากเว็บไซต์กรมสรรพากร www.rd.go.th 3. การแสดงเจตนา บริจาคเงินภาษีให้แก่ พรรคการเมืองตาม 2. ห้าม มิให้นำไป หักเป็นค่าลดหย่อน ตามมาตรา 47 แห่งประมวลรัษฎากร 4. ผู้มีเงินได้ ซึ่งมีสิทธิระบุการแสดงเจตนา บริจาคเงินภาษี ให้แก่พรรคการเมือง ในแบบ ภ.ง.ด.90 หรือ ภ.ง.ด.91 ให้ถือตามหลักเกณฑ์ ดังนี้ ( 1 ) กรณีสามีหรือภริยามีเงินได้ฝ่ายเดียว ให้ฝ่ายที่มีเงินได้เป็นผู้ระบุความประสงค์ในแบบฯ ( 2 ) กรณีสามี หรือภริยา ต่างฝ่ายต่างมีเงินได้ และแยกยื่นแบบฯ หรือรวมยื่นแบบฯ แต่แยกคำนวณภาษี ให้สามี และภริยา ต่างฝ่ายต่าง มีสิทธิระบุความประสงค์ในแบบฯ ( 3 ) กรณีสามี หรือภริยา ต่างฝ่ายต่างมีเงินได้ และรวมยื่นแบบฯ รวมทั้ง รวมคำนวณภาษี ให้ต่างฝ่าย ต่างมีสิทธิ ระบุความประสงค์ ของตนเอง ในแบบฯ โดยกรอก รายละเอียด การคำนวณ แยกรายการบุคคลใน 'ใบแนบ ภ.ง.ด.90/91 ปีภาษี 2551 รายละเอียด คำนวณ ภ.ง.ด.90/91 แยกรายบุคคล กรณีคู่สมรส รวมคำนวณภาษี และบริจาคภาษี แก่พรรคการเมือง' (แล้วแต่กรณี) ซึ่งสามารถ Download ได้จาก เว็บไซต์กรมสรรพากร HOT MENU > Download > บริการอิเล็กทรอนิกส์ > แบบแสดงรายการภาษี แบบคำร้อง/คำขอต่างๆ > ภาษีเงินได้บุคคลธรรมดา 'อนึ่ง หากผู้มีเงินได้ คำนวณภาษี ตามแบบ ภ.ง.ด.90 หรือ ภ.ง.ด.91 แล้ว มีเงินภาษี ที่ต้องชำระ ตั้งแต่ 100 บาท ขึ้นไป และมีภาษีหัก ณ ที่จ่าย ไว้เกินกว่า เงินภาษีที่ต้องชำระ เมื่อแจ้งความประสงค์ ขอคืนเงินภาษี ที่ชำระไว้เกิน พร้อมทั้ง แสดงเจตนา บริจาคเงินภาษี ให้แก่พรรคการเมืองด้วย กรมสรรพากร จะหักเงินบริจาคดังกล่าว จากเงินภาษี ที่ต้องชำระ ดังนั้น การบริจาค เงินภาษีดังกล่าว ไม่มีผลต่อเงินภาษี ที่ขอคืนไว้แต่อย่างใด ปรับปรุงล่าสุด: 21-11-2020",
    },
    {
        "id": 13,
        "question": "หากยื่นแบบแล้วมีภาษีต้องชำระจะขอผ่อนชำระภาษีได้หรือไม่ ?",
        "answer": "ถ้ามีภาษีที่ต้องชำระจำนวนตั้งแต่ 3,000 บาทขึ้นไป ทั้งภาษีครึ่งปีและภาษีสิ้นปี ผู้เสียภาษีมีสิทธิ ขอผ่อนชำระภาษีได้เป็น 3 งวดเท่า ๆ กัน โดยไม่ต้องเสียเงินเพิ่มใดๆ ผู้เสียภาษีอาจติดต่อขอผ่อนชำระได้ที่ สำนักงานสรรพากรพื้นที่สาขาโดยใช้แบบบ.ช. 35 จำนวน 1 ชุด 3 แผ่น ข้อความเหมือนกันดังนี้ งวดที่ 1 ชำระพร้อมกับการยื่นแบบแสดงรายการภายในวันที่ 30 กันยายน หรือวันที่ 31 มีนาคม งวดที่ 2 ชำระภายใน 1 เดือนนับแต่วันที่ต้องชำระงวดที่ 1 งวดที่ 3 ชำระภายใน 1 เดือนนับแต่วันที่ต้องชำระงวดที่ 2 ถ้าภาษีงวดใดงวดหนึ่งมิได้ชำระภายในกำหนดเวลาดังกล่าว ผู้เสียภาษีหมดสิทธิที่จะชำระภาษีเป็น รายงวด และต้องเสียเงินเพิ่มในอัตราร้อยละ 1.5 ต่อเดือน หรือเศษของเดือนของเงินภาษีงวดที่เหลือ ปรับปรุงล่าสุด: 21-11-2020",
    },
    {
        "id": 14,
        "question": "กรณีหักลดหย่อนค่าอุปการะเลี้ยงดูบิดามารดา",
        "answer": "การหักลดหย่อนค่าอุปการะเลี้ยงดูบิดามารดา ดังนี้ เนื่องจากบุตรหลายคนร่วมกันอุปการะเลี้ยงดู บิดามารดา ซึ่งกฎหมายให้สิทธิหักลดหย่อนแก่บุตรเพียงคนเดียวเท่านั้นโดยใช้ แบบ ล.ย.03 ตัวอย่างเช่น บุตร 3 คน (ก. ข. และ ค. ) อุปการะเลี้ยงดูบิดามารดา ปีภาษี 2552 ก. ใช้สิทธิหักลดหย่อนทั้งบิดาและมารดา ส่วน ข. และ ค. จะไม่สามารถใช้สิทธิได้ หรือ ข. ใช้สิทธิหักลดหย่อนบิดา และ ค. ใช้สิทธิหักลดหย่อน มารดา ส่วน ก. จะไม่สามารถใช้สิทธิได้ หมายเหตุ บุตรแต่ละคนควรตกลงกันว่าใครจะเป็นผู้ใช้สิทธิ หากตกลงกันไม่ได้ บุตรผู้มีเงินได้ทุกคนไม่มีสิทธิหักลดหย่อน บิดามารดาต้องมีอายุตั้งแต่ 60 ปีขึ้นไป ตัวอย่างเช่น บิดามารดา เกิดปี พ.ศ. 2493 ปีภาษี 2552 บิดามารดา มีอายุ 59 ปี ไม่สามารถนำมาหักลดหย่อนได้ ตัวอย่างเช่น บิดามารดา เกิดปี พ.ศ. 2492 ปีภาษี 2552 บิดามารดา มีอายุ 60 ปี สามารถนำมาหักลดหย่อน บิดามารดาต้องไม่มีเงินได้พึงประเมินในปีภาษี ที่ขอหักลดหย่อนเกิน 30,000 บาท ขึ้นไป ตัวอย่างเช่น ถ้าบิดามารดามีเงินได้พึงประเมินคนละ 30,001 บาท ไม่สามารถนำมาหักลดหย่อนได้ ถ้าบิดามารดามีเงินได้พึงประเมินคนละ 30,000 บาท สามารถนำมาหักลดหย่อนได้ ถ้าต้องการรายละเอียดเพิ่มเติมท่านสามารถเข้าไปในเว็บไซต์ www.rd.go.th > อ้างอิง > ประกาศอธิบดีกรมสรรพากร > เกี่ยวกับภาษีเงินได้ฉบับที่ 136 “การหักลดหย่อน กรณีบริจาคเงินเพื่อสนับสนุนการศึกษา และหลักฐานการใช้สิทธิยกเว้นภาษีเงินได้เป็นจำนวนสองเท่า” ในการยื่นแบบแสดงรายการภาษีเงินได้บุคคลธรรมดา(ภ.ง.ด.90,91) พบข้อผิดพลาด ในการใช้สิทธิยกเว้นภาษีเงินได้เป็นจำนวนสองเท่าของรายจ่ายที่จ่ายไปเป็นค่าใช้จ่ายเพื่อสนับสนุนการศึกษา มีรายละเอียดข้อแนะนำดังนี้ การใช้สิทธิยกเว้นภาษีเงินได้เป็นจำนวนสองเท่า ต้องเป็นการบริจาคให้กับสถานศึกษาตามรายชื่อสถานศึกษาที่กระทรวงศึกษาธิการประกาศกำหนด และไม่เป็นการบริจาคเพื่อแลกเปลี่ยนผลประโยชน์ต่อผู้ใดโดยเฉพาะ ตัวอย่างเช่น การจ่ายค่าเทอมของผู้มีเงินได้หรือบุตร การซื้อวัสดุอุปกรณ์ ตำราเรียน ของผู้มีเงินได้หรือบุตร หรือ ค่าเล่าเรียนพิเศษบุตร เป็นต้น เงินที่บริจาคนำไปยกเว้นภาษีเงินได้ เป็นจำนวน 2 เท่าของรายจ่าย แต่ต้องไม่เกินร้อยละสิบของเงินได้พึงประเมิน การบริจาคให้แก่สถานศึกษา ต้องเป็นค่าใช้จ่ายสำหรับรายการ ดังต่อไปนี้ 3.1 จัดหาหรือจัดสร้างอาคาร อาคารพร้อมที่ดิน หรือที่ดิน ให้แก่ สถานศึกษาเพื่อใช้ประโยชน์ทางการศึกษา 3.2 จัดหาวัสดุอุปกรณ์เพื่อการศึกษา แบบเรียน ตำรา หนังสือวิชาการ สื่อ และเทคโนโลยีเพื่อการศึกษาตลอดจนวัสดุอุปกรณ์อื่นๆ ที่เกี่ยวข้อง กับการศึกษา 3.3 จัดหาครู อาจารย์ หรือผู้ทรงคุณวุฒิทางการศึกษา การประดิษฐ์ การพัฒนา การค้นคว้า หรือการวิจัย สำหรับนักเรียน นิสิต หรือ นักศึกษาของสถานศึกษา ต้องมีหลักฐานจากสถานศึกษา เป็นใบเสร็จรับเงินของสถานศึกษาเท่านั้น โดย “ สถานศึกษาต้องระบุ คำว่า “เงินบริจาคเพื่อ.... 3.1, 3.2, 3.3 เพื่อ สนับสนุนการศึกษา ตามพระราชกฤษฎีกาฯ (ฉบับที่ 420) พ.ศ. 2547 ” ซึ่งให้เป็นส่วนกลางของสถานศึกษา นั้น หากเป็นการบริจาคให้แก่สถานศึกษา โดยมีวัตถุประสงค์ ที่นอกเหนือจาก ข้อ 3.1 3.2 และ 3.3 ให้นำไปหักลดหย่อนได้ ตามมาตรา 47(7)(ก) แห่งประมวลรัษฎากร ท่านสามารถค้นหารายชื่อสถานศึกษาตามข้อ 1 จากเว็บไซต์ กรมสรรพากร www.rd.go.th > กฎหมายภาษี > ประมวลรัษฎากร >ประกาศอธิบดีกรมสรรพากร แล้วคลิกที่รายชื่อสถานศึกษาที่กระทรวงศึกษาธิการประกาศ หรือสอบถามได้ที่สำนักงานสรรพากรพื้นที่(ทุกพื้นที่)/สำนักงานสรรพากรพื้นที่สาขา(ทุกสาขา) สำหรับพระราชกฤษฎีกา จากเว็บไซต์ กรมสรรพากร www.rd.go.th > กฎหมายภาษี > ประมวลรัษฎากร > พระราชกฤษฎีกา (ฉบับที่ 420) พ.ศ. 2547 ซึ่งแก้ไขเพิ่มเติม (ฉบับที่ 476) พ.ศ. 2551 “กรณีการหักลดหย่อนการอุปการะเลี้ยงดูคนพิการและคนทุพพลภาพ” ผู้มีเงินได้หักค่าลดหย่อนค่าอุปการะเลี้ยงดูคนพิการหรือคนทุพพลภาพได้คนละ 60,000 บาท ตามรายละเอียด ดังนี้ 1. ความสัมพันธ์กับผู้มีเงินได้ (1) เป็นบิดามารดาของผู้มีเงินได้ (2) เป็นบิดามารดาของสามีหรือภริยาของผู้มีเงินได้ (3) เป็นสามีหรือภริยาของผู้มีเงินได้ (4) เป็นบุตรชอบด้วยกฎหมายหรือบุตรบุญธรรมของผู้มีเงินได้ (5) เป็นบุตรชอบด้วยกฎหมายของสามีหรือภริยาของผู้มีเงินได้ (6) เป็นบุคคลอื่นที่ไม่ได้มีความสัมพันธ์กับผู้มีเงินได้ตาม (1) – (5) แต่ผู้มีเงินได้เป็นผู้อุปการะเลี้ยงดู ผู้มีเงินได้นำมาลดหย่อนได้ 1 คน 2. หลักเกณฑ์และองค์ประกอบการหักลดหย่อนค่าอุปการะเลี้ยงดูคนพิการ หลักเกณฑ์การเป็นคนพิการที่ผู้มีเงินได้จะหักลดหย่อนค่าอุปการะเลี้ยงดูคนพิการได้ (1) คนพิการต้องมีบัตรประจำตัวคนพิการตามกฎหมายว่าด้วยการส่งเสริมและพัฒนาคุณภาพชีวิตคนพิการ (2) เป็นบุคคลที่มีความสัมพันธ์กับผู้มีเงินได้ตาม 1 (3) ผู้มีเงินได้เป็นผู้อุปการะเลี้ยงดู และมีชื่อเป็นผู้ดูแลคนพิการในบัตรประจำตัวคนพิการ (4) คนพิการมีเงินได้พึงประเมินไม่เกิน 30,000 บาท ในปีภาษีที่ผู้มีเงินได้ใช้สิทธิหักลดหย่อน (ไม่รวมเงินได้ที่ได้รับยกเว้นตามมาตรา 42) (5) คนพิการต้องมีเลขประจำตัวประชาชน 13 หลัก (6) กรณีคนพิการมีผู้อุปการะเลี้ยงดูหลายคน ใครเป็นผู้มีสิทธิหักลดหย่อนนั้น ให้ดูว่าผู้มีเงินได้คนใดมีชื่อเป็นผู้ดูแลในบัตรประจำตัวคนพิการ ผู้มีเงินได้คนนั้นเป็นผู้มีสิทธิหักลดหย่อน 3. หลักเกณฑ์และองค์ประกอบการหักลดหย่อนค่าอุปการะเลี้ยงดูคนทุพพลภาพ กรณีคนทุพพลภาพที่ผู้มีเงินได้ซึ่งอุปการะเลี้ยงดูคนทุพพลภาพจะนำมาหักลดหย่อนภาษีได้ต้องมีองค์ประกอบดังนี้ (1) ต้องมีใบรับรองแพทย์ที่ได้ตรวจ และแสดงความเห็นว่าบุคคลนั้น มีภาวะจำกัดหรือขาดความสามารถ ในการประกอบกิจวัตรหลักอันเป็นปกต ิเยี่ยงบุคคลทั่วไปอันเนื่องมาจาก สาเหตุทางปัญหาสุขภาพ หรือความเจ็บป่วยที่เป็นต่อเนื่อง มาไม่น้อยกว่า 180 วัน หรือทุพพลภาพ มาแล้วไม่น้อยกว่า 180 วัน (2) บุคคลทุพพลภาพต้องมีเงินได้พึงประเมินไม่เกิน 30,000 บาท ในปีภาษีที่ผู้มีเงินได้ใช้สิทธิ หักลดหย่อน (ไม่รวมเงินได้ที่ได้รับยกเว้นตามมาตรา 42) (3) บุคคลทุพพลภาพต้องมีเลขประจำตัวประชาชน 13 หลัก (4) ผู้มีเงินได้ต้องมีหนังสือรับรองการเป็นผู้อุปการะเลี้ยงดูคนทุพพลภาพ 4. การรับรองและหลักฐานกรณีคนทุพพลภาพ กรณีคนทุพพลภาพที่ผู้มีเงินได้อุปการะเลี้ยงดูจะนำมาหักลดหย่อนได้ หลักฐาน ที่แสดงว่าเป็น คนทุพพลภาพคือ ใบรับรองแพทย์ กรณีหักลดหย่อนคนทุพพลภาพ นอกจากหลักฐานใบรับรองแพทย์แล้ว ผู้มีเงินได้ต้องมีหนังสือรับรอง การเป็นผู้อุปการะ เลี้ยงดูคนพิการ ที่รับรองว่าผู้มีเงินได้เป็นผู้อุปการะเลี้ยงดูคนทุพพลภาพ ผู้รับรองต้องมีความสัมพันธ์กับคนทุพพลภาพดังนี้ - สามี ภริยา - บุตรชอบด้วยกฎหมาย บุตรบุญธรรม หรือหลาน - บิดามารดา - พี่น้องร่วมบิดามารดาเดียวกัน หรือร่วมบิดาหรือร่วมมารดาเดียวกัน - ปู่ย่าตายาย - ลุงป้าน้าอา ผู้รับรองอาจเป็นบุคคลอื่นก็ได้ซึ่งได้แก่ - กำนันผู้ใหญ่บ้าน ในท้องที่ที่บุคคลทุพพลภาพอยู่อาศัย - บุคคลที่เป็นสมาชิกองค์กรปกครองส่วนท้องถิ่นในท้องที่ที่บุคคลทุพพลภาพอยู่อาศัย 5. การใช้สิทธิและหลักฐานการหักลดหย่อน (1) การหักค่าอุปการะเลี้ยงดูคนพิการ หรือคนทุพพลภาพให้หักได้ตลอดปีภาษี ไม่ว่ากรณีที่จะหักได้นั้น จะมีอยู่ตลอดปีภาษีหรือไม่ (2) กรณีผู้มีเงินได้มิได้เป็นผู้อยู่ในประเทศไทย ให้หักลดหย่อนค่าอุปการะเลี้ยงดูคนพิการหรือ คนทุพพลภาพได้เฉพาะคนพิการ หรือคนทุพพลภาพที่เป็นผู้อยู่ในประเทศไทย (3) กรณีผู้มีเงินได้หักลดหย่อนค่าอุปการะเลี้ยงดูคนพิการ และคนพิการดังกล่าวก็ยังเป็น ผู้ทุพพลภาพด้วยการใช้สิทธิหักค่าลดหย่อน ให้หักได้ในฐานะเป็นคนพิการฐานะเดียว (4) หลักฐานในการหักค่าลดหย่อน ผู้มีเงินได้ที่ใช้สิทธิหักค่าลดหย่อน อุปการะเลี้ยงดูคนพิการ คนทุพพลภาพต้องยื่นแบบ ล.ย.04 พร้อมหลักฐานดังต่อไปนี้ พร้อมกับการยื่นแบบแสดงรายการภาษี - กรณีหักลดหย่อนค่าอุปการะเลี้ยงดูคนพิการ ให้แนบภาพถ่ายบัตรประจำตัวคนพิการ โดยแนบในส่วนที่แสดงว่า ผู้มีเงินได้เป็นผู้ดูแลด้วย - กรณีหักลดหย่อนค่าอุปการะเลี้ยงดูคนทุพพลภาพให้แนบ (ก) ใบรับรองแพทย์ (ข) หนังสือรับรองการเป็นผู้อุปการะเลี้ยงดูคนทุพพลภาพ (แบบ ล.ย.04-1) (5) กรณีผู้มีเงินได้ซึ่งเป็นผู้มีสิทธิหักลดหย่อน ค่าอุปการะเลี้ยงดูบุตรซึ่งเป็นคนพิการหรือ คนทุพพลภาพ เป็นสามีหรือภริยา และความเป็นสามีภริยาได้มีอยู่ตลอดปี ภาษีโดยภริยามีเงินได้ตามมาตรา 40(1) และแยกยื่นแบบต่างหาก จากสามีให้สามีภริยาต่างฝ่ายต่างหักลดหย่อนได้คนละ 30,000 บาท ต่อบุตรที่พิการ หรือทุพพลภาพนั้น โดยสามีภริยาต้องแนบหลักฐานดังนี้ - ภาพถ่ายแบบ แบบ ล.ย.04 ของผู้มีเงินได้ ซึ่งสามีหรือภริยาของผู้มีเงินได้ ได้ลงลายมือชื่อรับรอง - ภาพถ่ายบัตรประจำตัวคนพิการ / ใบรับรองแพทย์และหนังสือการอุปการะเลี้ยงดู คนพิการ (ล.ย.04-1) ถ้าต้องการรายละเอียดเพิ่มเติมท่านสามารถเข้าไปในเว็บไซต์ https://www.rd.go.th > กฎหมายภาษี > ประกาศอธิบดีกรมสรรพากร > เกี่ยวกับภาษีเงินได้ฉบับที่ 182 https://www.rd.go.th > กฎหมายภาษี > ประกาศอธิบดีกรมสรรพากร > เกี่ยวกับภาษีเงินได้ฉบับที่ 193 “หลักเกณฑ์การยกเว้นการหักค่าลดหย่อนเบี้ยประกันชีวิตแบบบำนาญ” การยกเว้นภาษีเงินได้สำหรับการประกันชีวิตแบบบำนาญ เป็นการเพิ่มเติมค่าลดหย่อนเบี้ยประกันชีวิตที่มีอยู่แล้วตามปกติ 100,000 บาท เพิ่มขึ้นอีก 200,000 บาท ซึ่งวงเงินที่เพิ่มขึ้นต้องไม่เกินร้อยละ 15 ของเงินได้พึงประเมิน แต่เมื่อรวมกับเงินสะสมเข้ากองทุนสำรองเลี้ยงชีพ หรือเงินสะสมเข้ากองทุนบำเหน็จบำนาญข้าราชการ หรือเงินสะสมเข้ากองทุนสงเคราะห์ตามกฎหมายว่าด้วยโรงเรียนเอกชน แล้วแต่กรณีหรือเงินค่าซื้อหน่วยลงทุนในกองทุนรวมเพื่อการเลี้ยงชีพตามกฎหมายว่าด้วยหลักทรัพย์และตลาดหลักทรัพย์ ต้องไม่เกิน 500,000 บาท ในปีภาษีเดียวกัน หลักเกณฑ์การยกเว้นเงินได้สำหรับการประกันชีวิตแบบบำนาญ (1) เป็นกรมธรรม์ประกันชีวิตแบบบำนาญมีกำหนดเวลาตั้งแต่ 10 ปีขึ้นไป (2) ต้องเอาประกันไว้กับผู้รับประกันภัยที่ประกอบกิจการประกันชีวิตในไทย (3) มีการกำหนดการจ่ายผลประโยชน์เป็นรายงวดอย่างสม่ำเสมอ จะจ่ายเท่ากันทุกงวดหรือจ่ายในสัดส่วนที่เพิ่มขึ้นตามระยะเวลาการเอาประกันก็ได้ โดยจะจ่ายตามการทรงชีพที่อาจมีการรับรองจำนวนงวดในการจ่ายที่แน่นอน (4) มีการกำหนดช่วงอายุของการจ่ายผลประโยชน์เมื่อผู้มีเงินได้มีอายุตั้งแต่ 55 ปีขึ้นไป ถึงอายุ 85 ปี หรือกว่านั้น และต้องจ่ายเบี้ยประกันภัยครบถ้วนแล้ว ก่อนได้รับผลประโยชน์ ตัวอย่างการคำนวณฯ ที่ 1 ตัวอย่างที่ 1 นาย ข. มีเงินได้ปีละ 1,280,000 บาท จ่ายค่าเบี้ยประกันชีวิตแบบบำนาญปีละ 143,500 บาท จ่ายเงินสะสมเข้ากองทุนสำรองเลี้ยงชีพ 128,000 บาท และจ่ายค่าซื้อหน่วยลงทุนในกองทุน RMF จำนวน 121,000 บาท นาย ข. สามารถหักค่าลดหย่อนและยกเว้นภาษีได้ดังนี้ (1) คำนวณยอดการใช้สิทธิหักไม่เกินร้อยละ 15 ของเงินได้ แล้วพักไว้ = 1,280,000 x 15 % = 192,000 (2) นาย ข. จ่ายเบี้ยประกันชีวิตแบบบำนาญ 143,500 บาท ให้ไปใช้สิทธิหักประกันชีวิตปกติก่อน (10,000 + 90,000) = 100,000 บาท ส่วนที่เหลือนำไปใช้สิทธิค่าลดหย่อนประกันชีวิตแบบบำนาญ 43,500 บาท (143,500 - 100,000) (3) นำยอดค่าเบี้ยประกันชีวิตแบบบำนาญที่เหลือ 43,500 บาท เทียบกับวงเงินตาม 1. พบว่าไม่เกิน 192,000 บาท (15 % ของเงินได้และไม่เกิน 200,000 บาท) (4) นำยอดค่าประกันชีวิตแบบบำนาญที่เหลือ ไปรวมกับเงินสะสมเข้ากองทุนสำรองเลี้ยงชีพ และ เงินค่าซื้อหน่วยลงทุนในกองทุน RMF แล้วต้องไม่เกิน 500,000 บาท = 43,500 + 128,000 + 121,000 = 292,500 บาท นาย ข. สามารถนำเบี้ยประกันชีวิตแบบบำนาญไปหัก เป็นเบี้ยประกันชีวิตแบบปกติ 100,000 บาท และหักเบี้ยประกันชีวิตแบบบำนาญได้อีก 43,500 บาท ตัวอย่างการคำนวณฯ ที่ 2 ตัวอย่างที่ 2 นาย ง. มีเงินได้ปีละ 1,350,000 บาท จ่ายค่าเบี้ยประกันชีวิตแบบตลอดชีพปีละ 120,000 บาท ค่าเบี้ยประกันชีวิตแบบบำนาญปีละ 250,000 บาท จ่ายเงินสะสมเข้ากองทุนสำรองเลี้ยงชีพ 135,000 บาท และจ่ายค่าซื้อหน่วยลงทุนในกองทุน RMF จำนวน 100,000 บาท นาย ง. สามารถหักค่าลดหย่อนและยกเว้นภาษีได้ดังนี้ (1) คำนวณยอดการใช้สิทธิหักไม่เกินร้อยละ 15 ของเงินได้ แล้วพักไว้ = 1,350,000 x 15 % = 202,500 บาท (2) นาย ง. จ่ายเบี้ยประกันชีวิตแบบตลอดชีพ 120,000 บาท ให้นำเบี้ยประกันชีวิตแบบตลอดชีพไปใช้สิทธิหักเบี้ยประกันแบบปกติก่อน 100,000 บาท ส่วนที่เกิน 100,000 บาท(20,000 บาท) ตัดทิ้งเพราะหักเบี้ยประกันชีวิตแบบปกติครบถ้วนแล้ว (3) นำยอดเบี้ยประกันชีวิตแบบบำนาญที่จ่ายจริง 250,000 บาท เทียบกับวงเงินตาม 1. พบว่าเกิน 15% (202,500 บาท) และเกิน 200,000 บาท จึงสามารถใช้สิทธิหักลดหย่อนได้เต็ม 200,000 บาท (4) นำยอดค่าเบี้ยประกันภัยแบบบำนาญที่สามารถหักได้ตาม 3. ไปรวมกับเงินสะสมเข้ากองทุนสำรองเลี้ยงชีพและเงินค่าซื้อหน่วยลงทุนในกองทุน RMF แล้วต้องไม่เกิน 500,000 บาท = 200,000 + 135,000 + 100,000 = 435,000 บาท พบว่าไม่เกิน นาย ง. สามารถหักเบี้ยประกันชีวิตแบบตลอดชีพได้ 100,000 บาท และหักเบี้ยประกันชีวิตแบบบำนาญได้ 200,000 บาท ตัวอย่างการคำนวณฯ ที่ 3 ตัวอย่างที่ 3 นาย ก. มีเงินได้มีเงินได้ทุกประเภทภาษีรวมทั้งปีเท่ากับ 3,400,000 บาท ได้ซื้อประกันชีวิตแบบสะสมทรัพย์ของตนเองไว้รวมปีละ 60,000 บาท ได้ซื้อประกันชีวิตแบบบำนาญไว้ 300,000 บาท และซื้อหน่วยลงทุน RMF ไว้ในปีเดียวกันถึง 360,000 บาท นาย ก. สามารถหักลดหย่อนเบี้ยประกันชีวิตแบบบำนาญได้ดังนี้ (1) คำนวณยอดการใช้สิทธิหักไม่เกินร้อยละ 15 ของเงินได้ แล้วพักไว้มีเงินได้ทั้งปี 3,400,000 บาท ไม่เกินร้อยละ 15 มีเพดานสูงสุดเท่ากับ = 3,400,000 X 15% = ไม่เกิน 510,000 บาท (2) นาย ก. จ่ายเบี้ยประกันชีวิตแบบสะสมทรัพย์ 60,000 บาท ให้นำเบี้ยประกันไปใช้สิทธิหักเบี้ยประกันชีวิตปกติก่อน 60,000 บาท คงเหลืออีก 40,000 บาท (100,000 - 60,000 = 40,000 ) จึงนำเบี้ยประกันชีวิตแบบบำนาญมาใช้สิทธิให้ครบ 100,000 บาท (3) นาย ก. จ่ายเบี้ยประกันชีวิตแบบบำนาญ 300,000 บาท คงเหลืออีก 260,000 บาท (300,000 - 40,000) นำมาเปรียบเทียบกับตาม 1. พบว่าไม่เกิน จึงสามารถใช้สิทธิหักเบี้ยประกันชีวิตแบบบำนาญได้เต็ม 200,000 บาท (4) นำเบี้ยประกันชีวิตแบบบำนาญ 200,000 บาท ไปรวมกับเงินค่าซื้อหน่วยลงทุน RMF 360,000 บาท = 560,000 (200,000 + 360,000) ต้องไม่เกิน 500,000 บาท พบว่า เกินวงเงินที่สามารถหักได้รวม 60,000 บาท (560,000 - 500,000 = 60,000) (5) นำสิทธิที่สามารถหักเบี้ยประกันแบบบำนาญตาม 3. 200,000 บาท มาหักกับวงเงินที่เกินสิทธิหักได้ 60,000 บาท คงเหลือสิทธิที่หักได้จริง 140,000 บาท (200,000 – 60,000 = 140,000 บาท ) นาย ก. สามารถหักลดหย่อนเบี้ยประกันชีวิตแบบบำนาญได้ 180,000 บาท โดยนำไปหักลดหย่อนในส่วนของเงินประกันชีวิตปกติ 40,000 บาท รวมกับเบี้ยประกันชีวิตแบบสะสมทรัพย์ 60,000 บาท รวมเป็นเบี้ยประกันชีวิตปกติ 100,000 บาท และในส่วนของเบี้ยประกันชีวิตแบบบำนาญอีก 140,000 บาท ตัวอย่างการคำนวณฯ ที่ 4 ตัวอย่างที่ 4 นาย จ. มีเงินได้ปีละ 1,280,000 บาท จ่ายค่าเบี้ยประกันชีวิตแบบบำนาญปีละ 143,500 บาท จ่ายเงินสะสมเข้ากองทุนสำรองเลี้ยงชีพ 128,000 บาท และจ่ายค่าซื้อหน่วยลงทุนในกองทุน RMF จำนวน 121,000 บาท สมรสกับนาง เอ. มีเงินได้ปีละ 1,100,000 บาท ซึ่งได้จ่ายเบี้ยประกันชีวิตแบบบำนาญปีละ 140,000 บาท จ่ายเงินสะสมเข้ากองทุนสำรองเลี้ยงชีพ 120,000 บาท และจ่ายค่าซื้อหน่วยลงทุนในกองทุน RMF จำนวน 100,000 บาท นาง เอ. ใช้สิทธิแยกยื่นเสียภาษี นาย จ. และนาง เอ. สามารถหักค่าลดหย่อนและยกเว้นภาษีได้ดังนี้ การคำนวณของ นาย จ. (1) คำนวณยอดการใช้สิทธิหักไม่เกินร้อยละ 15 ของเงินได้ แล้วพักไว้ = 1,280,000 x 15 % = 192,000 (2) นาย จ. จ่ายเบี้ยประกันชีวิตแบบบำนาญ 143,500 บาท ให้ไปใช้สิทธิหักประกันชีวิตปกติก่อน (10,000 + 90,000) = 100,000 บาท ส่วนที่เหลือนำไปใช้สิทธิค่าลดหย่อนประกันชีวิตแบบบำนาญ 43,500 บาท (143,500 - 100,000) (3) นำยอดค่าเบี้ยประกันชีวิตแบบบำนาญที่เหลือ 43,500 บาท เทียบกับวงเงินตาม 1. พบว่าไม่เกิน 192,000 บาท (15 % ของเงินได้และไม่เกิน 200,000 บาท) (4) นำยอดค่าประกันชีวิตแบบบำนาญที่เหลือ ไปรวมกับเงินสะสมเข้ากองทุนสำรองเลี้ยงชีพ และ เงินค่าซื้อหน่วยลงทุนในกองทุน RMF แล้วต้องไม่เกิน 500,000 บาท = 43,500 + 128,000 + 121,000 = 292,500 บาท การคำนวณของ นางเอ. (1) คำนวณยอดการใช้สิทธิหักไม่เกินร้อยละ 15 ของเงินได้ แล้วพักไว้ = 1,100,000 x 15 % = 165,000 (2) นาย เอ. จ่ายเบี้ยประกันชีวิตแบบบำนาญ 140,000 บาท ให้ไปใช้สิทธิหักประกันชีวิตปกติก่อน (10,000 + 90,000) = 100,000 บาท ส่วนที่เหลือนำไปใช้สิทธิค่าลดหย่อนประกันชีวิตแบบบำนาญ 40,000 บาท (140,000 - 100,000) (3) นำยอดค่าเบี้ยประกันชีวิตแบบบำนาญที่เหลือ 40,000 บาท เทียบกับวงเงินตาม 1. พบว่าไม่เกิน 165,000 บาท (15 % ของเงินได้และไม่เกิน 200,000 บาท) (4) นำยอดค่าประกันชีวิตแบบบำนาญที่เหลือ ไปรวมกับเงินสะสมเข้ากองทุนสำรองเลี้ยงชีพ และ เงินค่าซื้อหน่วยลงทุนในกองทุน RMF แล้วต้องไม่เกิน 500,000 บาท = 40,000 + 120,000 + 100,000 = 260,000 บาท นาย จ. สามารถนำเบี้ยประกันชีวิตแบบบำนาญไปหัก เป็นเบี้ยประกันชีวิตแบบปกติ 100,000 บาทและหักเบี้ยประกันชีวิตแบบบำนาญได้อีก 43,500 บาท นาย เอ. สามารถนำเบี้ยประกันชีวิตแบบบำนาญไปหัก เป็นเบี้ยประกันชีวิตแบบปกติ 100,000 บาท และหักเบี้ยประกันชีวิตแบบบำนาญได้อีก 40,000 บาท ถ้าต้องการรายละเอียดเพิ่มเติมท่านสามารถเข้าไปในเว็บไซต์ https://www.rd.go.th > อ้างอิง > ประกาศอธิบดีกรมสรรพากร > เกี่ยวกับภาษีเงินได้ฉบับที่ 112 https://www.rd.go.th > อ้างอิง > ประกาศอธิบดีกรมสรรพากร > เกี่ยวกับภาษีเงินได้ฉบับที่ 172 https://www.rd.go.th > อ้างอิง > ประกาศอธิบดีกรมสรรพากร > เกี่ยวกับภาษีเงินได้ฉบับที่ 194 https://www.rd.go.th > อ้างอิง > กฎกระทรวง > ฉบับที่ 126 พ.ศ. 2509 ซึ่งแก้ไขเพิ่มเติม (ฉบับที่ 266) พ.ศ. 2551 ตารางเปรียบเทียบหลักเกณฑ์และวิธีการเพื่อการยกเว้นภาษีเงินได้ตามประกาศอธิบดีกรมสรรพากร เกี่ยวกับภาษีเงินได้ ปรับปรุงล่าสุด: 21-11-2020",
    },
    {
        "id": 15,
        "question": "ผู้มีเงินได้ทุกคนหักลดหย่อนภาษีอะไรได้บ้าง?",
        "answer": "ค่าลดหย่อนและยกเว้นเป็นอีกช่องหนึ่งที่กฎหมายกำหนดให้ นำไปหักออกจากเงินได้ก่อนนำมาคำนวณภาษีจ่ายได้แล้ว โดยมีหลักเกณฑ์การลดหย่อนต่างๆ แตกต่างกันออกไป สรุปได้ดังนี้  **(1) กรณีบุคคลธรรมดา หรือผู้มีเงินได้ตามมาตรา 40 แห่งประมวลรัษฎากร**  1. ผู้มีเงินได้ 60,000 บาท  2. คู่สมรส (ไม่มีเงินได้) 60,000 บาท\n3. ผู้มีเงินได้หรือคู่สมรสต่างฝ่ายต่างมีเงินได้ หักลดหย่อนรวมกันได้ ไม่เกิน 120,000 บาท\n4. บุตรชอบด้วยกฎหมายและบุตรบุญธรรม คนละ 30,000 บาท บุตรคนที่ 2 เป็นต้นไปที่เกิดตั้งแต่ปี 2561 หักหย่อนได้คนละ 60,000 บาท\n5. ค่าฝากครรภ์และค่าคลอดบุตร หักลดหย่อนเท่าที่จ่ายจริง แต่ไม่เกิน 60,000 บาท ในแต่ละคราว\n6. ค่าอุปการะเลี้ยงดูบิดามารดาที่มีอายุ 60 ปีขึ้นไป และอยู่ในความอุปการะเลี้ยงดูของผู้มีเงินได้ โดยบิดามารดาต้องมีเงินได้พึงประเมินในปีภาษีที่ขอหักลดหย่อนไม่เกิน 30,000 บาท หักค่าลดหย่อน คนละ 30,000 บาท และสามารถหักลดหย่อนสำหรับบิดามารดาของคู่สมรสที่ไม่มีเงินได้อีกคนละ 30,000 บาท\n7. ค่าอุปการะเลี้ยงดูคนพิการหรือคนทุพพลภาพ หักลดหย่อน คนละ 60,000 บาท\n8. ค่าเบี้ยประกันชีวิต (กรมธรรม์อายุ 10 ปีขึ้นไป) ของผู้มีเงินได้หักหย่อนและได้รับยกเว้นภาษีเงินได้สำหรับเงินได้ที่จ่ายจริงแต่ไม่เกิน 100,000 บาท กรมธรรม์ที่ได้รับผลประโยชน์ตอบแทนคืนคืน ต้องไม่เกินร้อยละ 20 ของเบี้ยประกันรายปี ทั้งนี้ หากคู่สมรสมีการประกันชีวิตและความคุ้มครองเสียค่าใช้จ่ายอยู่ ตลอดปีภาษี ผู้มีเงินได้มีสิทธิหักลดหย่อน สำหรับเบี้ยประกันชีวิตของคู่สมรสที่ไม่มีเงินได้หักลดหย่อน เท่าที่จ่ายจริง แต่ไม่เกิน 10,000 บาท\n9. ค่าเบี้ยประกันสุขภาพของผู้มีเงินได้ หักลดหย่อนเท่าที่จ่ายจริง แต่ไม่เกิน 25,000 บาท ทั้งนี้ หากรวมกับค่าเบี้ยประกันชีวิต ต้องไม่เกิน 100,000 บาท\n10. ค่าเบี้ยประกันสุขภาพบิดามารดาของผู้มีเงินได้และคู่สมรส หักลดหย่อนเท่าที่จ่ายจริง แต่ไม่เกิน 15,000 บาท ทั้งนี้ บิดามารดาของผู้มีเงินได้และคู่สมรสต้องไม่มีเงินได้พึงประเมินในปีภาษีที่ใช้สิทธิยกเว้นภาษีเงินได้เกิน 30,000 บาท\n11. เงินสะสมที่จ่ายเข้ากองทุนสำรองเลี้ยงชีพ หักลดหย่อนเท่าที่จ่ายจริง แต่ไม่เกิน 10,000 บาท\n12. เงินค่าซื้อหน่วยลงทุนในกองทุนรวมเพื่อการเลี้ยงชีพ ได้รับยกเว้นเท่าที่จ่ายค่าซื้อหน่วยลงทุนในกองทุนรวมเพื่อการเลี้ยงชีพตามกฎหมายว่าด้วยหลักทรัพย์และตลาดหลักทรัพย์ ในอัตราร้อยไม่เกินละ 30 ของเงินได้พึงประเมินที่ได้รับซึ่งต้องเสียภาษีเงินได้ในปีภาษีนั้น  ต้องถือหน่วยลงทุนไม่น้อยกว่า 5 ปีนับแต่วันที่ซื้อหน่วยลงทุนครั้งแรก และเมื่อรวมกับเงินสะสมเข้ากองทุนสำรองเลี้ยงชีพ เงินสะสมเข้ากองทุนบำเหน็จบำนาญข้าราชการ เงินสะสมเข้ากองทุนสงเคราะห์ตามกฎหมายว่าด้วยโรงเรียนเอกชน แต่ต้องไม่เกิน 500,000 บาท\n\n13. ค่าเบี้ยประกันชีวิตแบบบำนาญ หักลดหย่อนเท่าที่จ่ายจริง ไม่เกินร้อยละ 15 ของเงินได้ แต่ไม่เกิน 200,000 บาท\nทั้งนี้ ต้องเป็นค่าเบี้ยประกันชีวิตแบบบำนาญ ความคุ้มครองตั้งแต่ 10 ปีขึ้นไป และจ่ายผลประโยชน์ บำนาญเมื่อผู้มีเงินได้อายุตั้งแต่ 55 ปีขึ้นไปถึงอายุ  85 ปีหรือกว่านั้น และเมื่อรวมกับเงินสะสมเข้ากองทุน สำรองเลี้ยงชีพ เงินสะสมเข้ากองทุนบำเหน็จบำนาญข้าราชการ (กบข.) เงินสะสมเข้ากองทุนสงเคราะห์ตาม กฎหมายว่าด้วยโรงเรียนเอกชน  เงินค่าซื้อหน่วยลงทุนรวมเพื่อการเลี้ยงชีพ ต้องไม่เกิน 500,000 บาท\n14. เงินสะสม กบข. หักลดหย่อนเท่าที่จ่ายจริง แต่ไม่เกิน 500,000 บาท\n15. เงินสะสมกองทุนสงเคราะห์ครูโรงเรียนเอกชน หักลดหย่อนเท่าที่จ่ายจริง แต่ไม่เกิน 500,000 บาท\n16. เงินสะสมกองทุนการออมแห่งชาติ (กอช.) หักลดหย่อนเท่าที่จ่ายจริง  แต่ไม่เกิน 30,000 บาท\n17. ค่าซื้อหน่วยลงทุนในกองทุนรวมเพื่อการออม SSF หักลดหย่อนเท่าที่จ่ายจริง  แต่ไม่เกินร้อยละ 30 ของเงินได้พึงประเมิน ต้องไม่เกิน 200,000 บาท\n18. ดอกเบี้ยกู้ยืมที่จ่ายให้แก่ธนาคารหรือสถาบันการเงินอื่น บริษัทประกันชีวิต สหกรณ์ หรือนายจ้างสำหรับการกู้ยืมเงินเพื่อ เช่าซื้อ หรือสร้างอาคารอยู่อาศัย โดยเงินค่าอาคารที่เช่าหรือสร้างเป็นประโยชน์การกู้ยืม หักค่าลดหย่อนเท่าที่จ่ายจริง แต่ไม่เกิน 100,000 บาท\n19. เงินสมทบประกันสังคม หักลดหย่อนเท่าที่จ่ายจริง ไม่เกิน 9,000 บาท\n20. เงินลงทุนในหุ้น หรือการเป็นหุ้นส่วนเพื่อจัดตั้ง หรือเพิ่มทุนบริษัท หรือห้างหุ้นส่วนนิติบุคคลที่ได้รับ จดทะเบียนวิสาหกิจเพื่อสังคมและได้จดแจ้งการเป็นวิสาหกิจเพื่อสังคม หักลดหย่อนเท่าที่จ่ายจริง แต่ไม่เกิน 100,000 บาท\n21. ค่าซื้อหน่วยลงทุนในกองทุนรวมไทยเพื่อความยั่งยืน (Thai ESG) หักลดหย่อนเท่าที่จ่ายจริง ไม่เกินร้อยละ 30 ของเงินได้พึงประเมิน แต่ต้องไม่เกิน 300,000 บาท ต้องถือหน่วยลงทุนต่อเนื่องกันไม่น้อยกว่า 5 ปีนับตั้งแต่วันซื้อหน่วยลงทุน\n22. ค่าซื้อสินค้า หรือค่าบริการ (Easy e-Receipt) ตั้งแต่วันที่ 1 มกราคม 2567 – 15 กุมภาพันธ์ 2567 หักลดหย่อนเท่าที่จ่ายจริง แต่ไม่เกิน 50,000 บาท\n23. ค่าก่อสร้างอาคารที่อยู่อาศัยในที่ดินรู้จก (สำหรับผู้ประกอบการจดทะเบียนภาษีมูลค่าเพิ่ม) หักลดหย่อนไม่เกิน 100,000 บาท (จำนวน  10,000 บาท ต่อทุกจำนวน 1,000,000 บาท)\n24. ค่าท่องเที่ยวภายในประเทศ (เที่ยวเมืองรอง) ตั้งแต่วันที่ 1 พฤษภาคม 2567 – 30 พฤศจิกายน 2567 หักลดหย่อนเท่าที่จ่ายจริง แต่ไม่เกิน 15,000 บาท\n25. ค่าซ่อมบ้าน (จากอุทกภัยวันที่ 16 สิงหาคม 2567 - 31 ธันวาคม 2567) หักลดหย่อนเท่าที่จ่ายจริง แต่ไม่เกิน 100,000 บาท\n26. ค่าซ่อมรถ (จากอุทกภัยวันที่ 16 สิงหาคม 2567   - 31 ธันวาคม 2567) หักลดหย่อนเท่าที่จ่ายจริง แต่ไม่เกิน 30,000 บาท\n\n27. เงินบริจาคให้แก่พรรคการเมือง เงิน ทรัพย์สิน หรือประโยชน์อื่นใดที่ให้เพื่อสนับสนุนการจัดกิจกรรม ระดมทุนของพรรคการเมือง หักลดหย่อนเท่าที่จ่ายจริง แต่ไม่เกิน 10,000 บาท\n- เงินบริจาค\n- เงินบริจาคสนับสนุนการศึกษา การกีฬา และโรงพยาบาลรัฐ ผ่านระบบบริจาคอิเล็กทรอนิกส์ (e-Donation) หักได้ 2 เท่าของที่จ่ายจริง แต่ไม่เกินร้อยละ 10 ของเงินได้หลังหักค่าใช้จ่ายและค่าลดหย่อน\n- เงินบริจาคอื่น หักได้เท่าที่จ่ายจริง แต่ไม่เกินร้อยละ 10 ของเงินได้หลังหักค่าใช้จ่ายและค่าลดหย่อน\n\n**(2) กรณีห้างหุ้นส่วนสามัญที่มิใช่นิติบุคคล หรือคณะบุคคลที่มิใช่นิติบุคคล หักลดหย่อนได้คนละ 60,000 บาท แต่รวมกันต้องไม่เกิน 120,000 บาท**\n\n**(3) กรณีกองมรดกที่ยังมิได้แบ่ง หักลดหย่อนได้ 60,000 บาท**",
    },
    {
        "id": 16,
        "question": "บัญชีอัตราภาษีเงินได้บุคคลธรรมดา เริ่มใช้ปีภาษี 2560",
        "answer": "บัญชีอัตราภาษีเงินได้บุคคลธรรมดา เริ่มใช้ปีภาษี 2560\n\n| เงินได้สุทธิ (บาท) | ช่วงเงินได้สุทธิ ของแต่ละขั้น | อัตราภาษี (ร้อยละ) |\n|---|---|---|\n| 1 - 150,000 | 150,000 | ได้รับยกเว้น |\n| 150,001 - 300,000 | 150,000 | 5 |\n| 300,001 - 500,000 | 200,000 | 10 |\n| 500,001 - 750,000 | 250,000 | 15 |\n| 750,001 - 1,000,000 | 250,000 | 20 |\n| 1,000,001 - 2,000,000 | 1,000,000 | 25 |\n| 2,000,001 - 5,000,000 | 3,000,000 | 30 |\n| 5,000,001 บาทขึ้นไป | - | 35 |\n\nหมายเหตุ : เงินได้สุทธิเฉพาะส่วนที่เกิน 150,000 บาท ยังคงได้รับยกเว้นตามพระราชกฤษฎีกา ว่าด้วยการยกเว้นรัษฎากร (ฉบับที่ 470) พ.ศ.2551",
    },
    {
        "id": 17,
        "question": "ยื่นแบบแสดงรายการได้ที่ไหน ?",
        "answer": "ยื่นแบบแสดงรายการได้ที่ไหน ?\n\n1. สำนักงานสรรพากรพื้นที่สาขา (เดิมเรียกว่า สำนักงานสรรพากรเขต/อำเภอ)\nสำหรับการยื่นแบบฯ ภ.ง.ด.90 หรือ ภ.ง.ด.91 ผู้มีเงินได้สามารถยื่นแบบฯ ณ สำนักงานสรรพากรพื้นที่\nสาขาทุกแห่ง\n\n2. ที่ทำการไปรษณีย์ สำหรับการยื่นแบบฯ ระหว่างวันที่ 1 มกราคม ถึงวันที่ 31 มีนาคม เท่านั้น มีหลักเกณฑ์\nดังนี้\n\n2.1 ผู้มีเงินได้มีภูมิลำเนาอยู่ในกรุงเทพมหานคร\n\n2.2 ยื่นแบบ ภ.ง.ด.90 หรือ ภ.ง.ด.91 โดยส่งทางไปรษณีย์ลงทะเบียน พร้อมแนบเช็ค (ประเภท ข. ค.\nหรือ ง.) หรือ ธนาณัติ (ตามจำนวนเงินภาษีที่ต้องชำระทั้งจำนวน) โดยส่งไปยัง\n\n\nกองบริหารการคลังและรายได้\nกรมสรรพากร อาคารกรมสรรพากร\nเลขที่ 90 ซอยพหลโยธิน 7 ถนนพหลโยธิน\nเขตพญาไท กรุงเทพฯ 10400\n\n\n2.3 กรมสรรพากรจะถือเอาวันที่ลงทะเบียนไปรษณีย์เป็นวันรับแบบและชำระภาษี และจะส่งใบเสร็จรับเงิน\nให้แก่ผู้ยื่นแบบฯ ทางไปรษณีย์ลงทะเบียน\n\n2.4 กรณีผู้มีเงินได้ที่มีภูมิลำเนา อยู่ในต่างจังหวัด หรือประสงค์จะขอ ชำระภาษีเป็นงวด จะยื่นแบบ ภ.ง.ด.\n90 หรือ ภ.ง.ด.91 โดยส่งทางไปรษณีย์ลงทะเบียนไม่ได้\n\n3. Internet ทาง Web Site ของกรมสรรพากร ที่ www.rd.go.th\nหรือจะยื่นผ่าน Rd smart tax application ทางโทรศัพท์มือถือ ก็ได้",
    },
    {
        "id": 18,
        "question": "เงินได้ที่ได้รับยกเว้นมีอะไรบ้างที่ได้รับยกเว้นภาษี?",
        "answer": "เงินได้ที่ได้รับยกเว้นภาษีมีอยู่หลายกรณีที่สำคัญๆ ได้แก่ การยกเว้นตามมาตรา 42 แห่งประมวลรัษฎากร การยกเว้นตามกฎกระทรวง ฉบับที่ 126 การยกเว้นตามพระราชกฤษฎีกาบางต่าง ๆ เป็นต้น เงินได้ที่ได้รับ ยกเว้นภาษีได้ดังนี้\n\n**เงินได้พึงประเมินที่ได้รับยกเว้นไม่ต้องนำมารวมคำนวณภาษี**\n\n(1) ค่าเบี้ยเลี้ยง หรือค่าพาหนะ ซึ่งลูกจ้าง หรือผู้รับหน้าที่หรือตำแหน่งงาน หรือผู้รับทำงานให้โดย จ่ายไปโดยสุจริตตามความจำเป็น เฉพาะในการที่จะต้องปฏิบัติการตามหน้าที่ของตน และได้จ่ายไปทั้งหมดในการนั้น\n\n(2) ค่าพาหนะ และเบี้ยเลี้ยงเดินทางตามอัตราที่รัฐบาลกำหนดไว้ โดยพระราชกฤษฎีกาว่าด้วยอัตรา ค่าพาหนะและเบี้ยเลี้ยงเดินทาง\n\n(3) เงินค่าเดินทางซึ่งนายจ้างจ่ายให้ลูกจ้าง เฉพาะส่วนที่ลูกจ้างได้จ่ายทั้งหมดโดยจำเป็นเพื่อการ เดินทางจากต่างถิ่นในการเข้ารับงานเป็นครั้งแรกหรือในการกลับภูมิลำเนาเดิมเมื่อการจ้างได้สิ้นสุดลงแล้ว แต่ข้อยกเว้น นี้ให้รวมถึงเงินค่าเดินทางที่ลูกจ้างได้รับในการกลับภูมิลำเนาเดิมและในการเข้ารับงานของนายจ้างเดิมภายใน 365 วัน นับแต่วันที่การจ้างครั้งก่อนได้สิ้นสุดลง\n\n(4) ในกรณีที่นายจ้าง และลูกจ้างมิได้ทำสัญญาผูกพันโดยตรงต่อกันตามระบบบัญชีลูกหนี้การเงินได้ ทุนการศึกษา 2475 มีข้อกำหนดว่านายจ้างชำระเงินทุน เงินเดือน ค่าธรรมเนียม เงินค่าหน่วยกิต หรือเงินปันผล ให้แก่ลูกจ้างเป็นจำนวนเดียวกับเงินหรือภารงานที่จ้างได้สิ้นสุดลงแล้ว แม้เงินมีจำนวนนับได้ชำระภายหลังที่ใช้ บทบัญญัติในส่วนนี้ก็ดี เงินบำเหน็จ เงินค่าธรรมเนียม เงินค่าหน่วยกิต หรือเงินปันผลส่วนที่เป็นค่าจ้างแรงงาน อันได้ทำในเวลาต่อมาให้พระราชบัญญัติภาษีเงินได้ พุทธศักราช 2475 นั้น ไม่ต้องรวมคำนวณเพื่อเสียภาษีสำหรับ\n\n(5) เงินเพิ่มพิเศษประจำตำแหน่ง และเงินค่าเช่าบ้าน หรือบ้านที่ให้อยู่โดยไม่ต้องเสียค่าเช่า สำหรับ ข้าราชการสถานทูต หรือสถานกงสุลไทยในต่างประเทศ\n\n(6) เงินได้จากมรดก หรือจากการให้โดยเสน่หา เพราะเหตุศิลธรรม จรรยา หรือตามธรรมเนียมประเพณี หรือจากการรับมรดกหรือรับจากายโอนกรรมสิทธิ์ หรือค่าตอบแทนที่ช่างทางราชการจ่ายให้\n\n(7) เบี้ยประชุมกรรมการหรือกรรมการสอบสวน ค่าสอบที่ทางราชการหรือสถานศึกษาของทาง ราชการจ่ายให้\n\n(8) ดอกเบี้ยดังต่อไปนี้\n\n    (ก) ดอกเบี้ยธนาคารออมสินของรัฐบาล เฉพาะเงินฝากออมสินของรัฐบาลประเภทฝากเผื่อเรียก\n    (ข) ดอกเบี้ยเงินกู้สหกรณ์ประเภทดอกเบี้ยที่ได้รับครั้งแรกเกษตรกร\n\n(9) ดอกเบี้ยเงินฝากประจำธนาคารในราชอาณาจักรที่ต้องจ่ายคืนเมื่อครบกำหนดเวลาฝากหรือเมื่อทวงถามเฉพาะกรณีที่ผู้มีเงินได้ได้รับดอกเบี้ยดังกล่าวในจำนวนรวมกันทั้งสิ้นไม่เกิน 10,000 บาทตลอดปีภาษี ทั้งนี้ ตามหลักเกณฑ์ วิธีการ และเงื่อนไขที่อธิบดีประกาศกำหนด\n\n(9) การขายสังหาริมทรัพย์อันเป็นมรดก หรือสังหาริมทรัพย์ที่ได้มาโดยมิได้มุ่งในทางการค้าหรือ หากกำไร แต่ไม่รวมถึงเรือกำปั่น เรือที่มีระวางตั้งแต่ 6 ตันขึ้นไป เรือกลไฟ หรือเรือยนต์ที่มีระวางตั้งแต่ 5 ตันขึ้นไปหรือแพ\n\n(10) เงินได้ที่ได้รับจากการรับมรดก\n\n(11) รางวัลเพื่อการศึกษาหรือค้นคว้าในวิทยาการ รางวัลสลากกินแบ่ง หรือสลากออมสินของรัฐบาล รางวัลที่ทางราชการจ่ายให้ในการประกวดหรือแข่งขัน ซึ่งผู้ได้รับมิใช่มีอาชีพในการประกวดหรือแข่งขัน หรือ สินบนรางวัลที่ทางราชการจ่ายให้เพื่อประโยชน์ในการปราบปรามการกระทำความผิด\n\n(12) บำนาญพิเศษ บำเหน็จพิเศษ บำเหน็จตกทอด หรือบำนาญพิเศษตกทอด\n\n(13) ค่าสินไหมทดแทนเพื่อละเมิด เงินได้จากการประกันภัย หรือจากการฌาปนกิจสงเคราะห์\n\n(14) ส่วนแบ่งของกำไรจากห้างหุ้นส่วนสามัญหรือคณะบุคคลที่มิใช่นิติบุคคล ซึ่งต้องเสียภาษีตาม บทบัญญัติในส่วนนี้ แต่ไม่รวมถึงเงินส่วนแบ่งของกำไรของกองทุนรวม (ยกเว้นโดย พระราชบัญญัติแก้ไข เพิ่มเติม (ฉบับที่ 39) พ.ศ. 2557 ณ วันที่ 26 ธันวาคม 2557)\n\n(15) เงินได้ของชาวนาที่ได้จากการขายข้าวอันเกิดจากการปลูกข้าวผสมผสานหรือครอบครัวทำไร่นาเอง",
    },
]

def map_records(records, model=model):
    cloned_records = records.copy()
    for rec in cloned_records:
        metadata = {
            'question': rec['question'],
            'answer': rec['answer'],
        }
        rec['id'] = str(rec['id'])
        # rec['question'] = model.encode(rec['question']).tolist()
        # rec['answer'] = model.encode(rec['answer']).tolist()
        rec['values'] = model.encode(f"{rec['question']}|{rec['answer']}").tolist()
        rec['metadata'] = metadata
        del rec['question']
        del rec['answer']
    return cloned_records

sample_records = map_records(sample_records)
sample_records[:3]


[{'id': '1',
  'values': [-3.312761691631749e-05,
   -0.0033086836338043213,
   -0.016734331846237183,
   -0.003538753604516387,
   -0.010135634802281857,
   -0.010973761789500713,
   0.020540772005915642,
   0.011061053723096848,
   0.024569012224674225,
   0.0010938704945147038,
   -0.0010470995912328362,
   -0.022309787571430206,
   -0.011204596608877182,
   0.009822030551731586,
   0.002851403085514903,
   0.006642857566475868,
   -0.03261769562959671,
   -0.013705344870686531,
   -0.04715555161237717,
   0.01801052875816822,
   0.0038616349920630455,
   -0.026649247854948044,
   -0.013585733249783516,
   -0.016234030947089195,
   -0.006353287026286125,
   -0.0009986417135223746,
   -0.020209260284900665,
   -0.014973713085055351,
   0.00648127868771553,
   0.012641395442187786,
   -0.004642677493393421,
   -0.019140884280204773,
   -0.005959727801382542,
   -0.023515425622463226,
   0.010958748869597912,
   0.004440366290509701,
   -0.026070212945342064,
   -0.0021431664936244488,

## DO NOT RUN IF NOT NEEDED ⚠️

In [57]:
# ⚠️ comment out if need to insert into the database
pc_index.upsert(vectors=sample_records)

# ⚠️ cleanup old rec records in the index
# pc_index.delete(delete_all=True)


{'upserted_count': 18}

## Making Queries

Now that our index is populated we can begin making queries. We are performing a semantic search for similar questions, so we should embed and search with another question. Let's begin.

In [58]:
def find_similar_questions(question, top_k=5):
    # Embed the question into a query vector
    xq = model.encode(question).tolist()

    # Now query Pinecone to find similar questions
    return pc_index.query(vector=xq, top_k=top_k, include_metadata=True)


In [59]:
question = "ภาษี 2568"
xc = find_similar_questions(question)
xc


{'matches': [{'id': '16',
              'metadata': {'answer': 'บัญชีอัตราภาษีเงินได้บุคคลธรรมดา '
                                     'เริ่มใช้ปีภาษี 2560\n'
                                     '\n'
                                     '| เงินได้สุทธิ (บาท) | ช่วงเงินได้สุทธิ '
                                     'ของแต่ละขั้น | อัตราภาษี (ร้อยละ) |\n'
                                     '|---|---|---|\n'
                                     '| 1 - 150,000 | 150,000 | ได้รับยกเว้น '
                                     '|\n'
                                     '| 150,001 - 300,000 | 150,000 | 5 |\n'
                                     '| 300,001 - 500,000 | 200,000 | 10 |\n'
                                     '| 500,001 - 750,000 | 250,000 | 15 |\n'
                                     '| 750,001 - 1,000,000 | 250,000 | 20 |\n'
                                     '| 1,000,001 - 2,000,000 | 1,000,000 | 25 '
                                     '|\n'
                           

In [60]:
def print_query_results(results):
    for result in results['matches']:
        print(f"{round(result['score'], 2)}: {result['metadata']['answer']}")

print_query_results(xc)


0.47: บัญชีอัตราภาษีเงินได้บุคคลธรรมดา เริ่มใช้ปีภาษี 2560

| เงินได้สุทธิ (บาท) | ช่วงเงินได้สุทธิ ของแต่ละขั้น | อัตราภาษี (ร้อยละ) |
|---|---|---|
| 1 - 150,000 | 150,000 | ได้รับยกเว้น |
| 150,001 - 300,000 | 150,000 | 5 |
| 300,001 - 500,000 | 200,000 | 10 |
| 500,001 - 750,000 | 250,000 | 15 |
| 750,001 - 1,000,000 | 250,000 | 20 |
| 1,000,001 - 2,000,000 | 1,000,000 | 25 |
| 2,000,001 - 5,000,000 | 3,000,000 | 30 |
| 5,000,001 บาทขึ้นไป | - | 35 |

หมายเหตุ : เงินได้สุทธิเฉพาะส่วนที่เกิน 150,000 บาท ยังคงได้รับยกเว้นตามพระราชกฤษฎีกา ว่าด้วยการยกเว้นรัษฎากร (ฉบับที่ 470) พ.ศ.2551
0.47: ถ้ามีภาษีที่ต้องชำระจำนวนตั้งแต่ 3,000 บาทขึ้นไป ทั้งภาษีครึ่งปีและภาษีสิ้นปี ผู้เสียภาษีมีสิทธิ ขอผ่อนชำระภาษีได้เป็น 3 งวดเท่า ๆ กัน โดยไม่ต้องเสียเงินเพิ่มใดๆ ผู้เสียภาษีอาจติดต่อขอผ่อนชำระได้ที่ สำนักงานสรรพากรพื้นที่สาขาโดยใช้แบบบ.ช. 35 จำนวน 1 ชุด 3 แผ่น ข้อความเหมือนกันดังนี้ งวดที่ 1 ชำระพร้อมกับการยื่นแบบแสดงรายการภายในวันที่ 30 กันยายน หรือวันที่ 31 มีนาคม งวดที่ 2 ชำระภายใน 1 เดือนนับแต่

In [61]:
question2 = "สามีภรรยาต้องเสียภาษีเท่าไร"

xc2 = find_similar_questions(question2)
print_query_results(xc2)


0.58: ค่าลดหย่อนและยกเว้นเป็นอีกช่องหนึ่งที่กฎหมายกำหนดให้ นำไปหักออกจากเงินได้ก่อนนำมาคำนวณภาษีจ่ายได้แล้ว โดยมีหลักเกณฑ์การลดหย่อนต่างๆ แตกต่างกันออกไป สรุปได้ดังนี้  **(1) กรณีบุคคลธรรมดา หรือผู้มีเงินได้ตามมาตรา 40 แห่งประมวลรัษฎากร**  1. ผู้มีเงินได้ 60,000 บาท  2. คู่สมรส (ไม่มีเงินได้) 60,000 บาท
3. ผู้มีเงินได้หรือคู่สมรสต่างฝ่ายต่างมีเงินได้ หักลดหย่อนรวมกันได้ ไม่เกิน 120,000 บาท
4. บุตรชอบด้วยกฎหมายและบุตรบุญธรรม คนละ 30,000 บาท บุตรคนที่ 2 เป็นต้นไปที่เกิดตั้งแต่ปี 2561 หักหย่อนได้คนละ 60,000 บาท
5. ค่าฝากครรภ์และค่าคลอดบุตร หักลดหย่อนเท่าที่จ่ายจริง แต่ไม่เกิน 60,000 บาท ในแต่ละคราว
6. ค่าอุปการะเลี้ยงดูบิดามารดาที่มีอายุ 60 ปีขึ้นไป และอยู่ในความอุปการะเลี้ยงดูของผู้มีเงินได้ โดยบิดามารดาต้องมีเงินได้พึงประเมินในปีภาษีที่ขอหักลดหย่อนไม่เกิน 30,000 บาท หักค่าลดหย่อน คนละ 30,000 บาท และสามารถหักลดหย่อนสำหรับบิดามารดาของคู่สมรสที่ไม่มีเงินได้อีกคนละ 30,000 บาท
7. ค่าอุปการะเลี้ยงดูคนพิการหรือคนทุพพลภาพ หักลดหย่อน คนละ 60,000 บาท
8. ค่าเบี้ยประกันชีวิต (กรมธรรม์อายุ 10 ปีขึ้น

In [62]:
question3 = 'มีคู่สมรสหักลดหย่อนภาษีได้มากสุดเท่าไร'
xc3 = find_similar_questions(question3)
print_query_results(xc3)


0.67: ค่าลดหย่อนและยกเว้นเป็นอีกช่องหนึ่งที่กฎหมายกำหนดให้ นำไปหักออกจากเงินได้ก่อนนำมาคำนวณภาษีจ่ายได้แล้ว โดยมีหลักเกณฑ์การลดหย่อนต่างๆ แตกต่างกันออกไป สรุปได้ดังนี้  **(1) กรณีบุคคลธรรมดา หรือผู้มีเงินได้ตามมาตรา 40 แห่งประมวลรัษฎากร**  1. ผู้มีเงินได้ 60,000 บาท  2. คู่สมรส (ไม่มีเงินได้) 60,000 บาท
3. ผู้มีเงินได้หรือคู่สมรสต่างฝ่ายต่างมีเงินได้ หักลดหย่อนรวมกันได้ ไม่เกิน 120,000 บาท
4. บุตรชอบด้วยกฎหมายและบุตรบุญธรรม คนละ 30,000 บาท บุตรคนที่ 2 เป็นต้นไปที่เกิดตั้งแต่ปี 2561 หักหย่อนได้คนละ 60,000 บาท
5. ค่าฝากครรภ์และค่าคลอดบุตร หักลดหย่อนเท่าที่จ่ายจริง แต่ไม่เกิน 60,000 บาท ในแต่ละคราว
6. ค่าอุปการะเลี้ยงดูบิดามารดาที่มีอายุ 60 ปีขึ้นไป และอยู่ในความอุปการะเลี้ยงดูของผู้มีเงินได้ โดยบิดามารดาต้องมีเงินได้พึงประเมินในปีภาษีที่ขอหักลดหย่อนไม่เกิน 30,000 บาท หักค่าลดหย่อน คนละ 30,000 บาท และสามารถหักลดหย่อนสำหรับบิดามารดาของคู่สมรสที่ไม่มีเงินได้อีกคนละ 30,000 บาท
7. ค่าอุปการะเลี้ยงดูคนพิการหรือคนทุพพลภาพ หักลดหย่อน คนละ 60,000 บาท
8. ค่าเบี้ยประกันชีวิต (กรมธรรม์อายุ 10 ปีขึ้น

In [63]:
xc3['matches']


[{'id': '15',
  'metadata': {'answer': 'ค่าลดหย่อนและยกเว้นเป็นอีกช่องหนึ่งที่กฎหมายกำหนดให้ '
                         'นำไปหักออกจากเงินได้ก่อนนำมาคำนวณภาษีจ่ายได้แล้ว '
                         'โดยมีหลักเกณฑ์การลดหย่อนต่างๆ แตกต่างกันออกไป '
                         'สรุปได้ดังนี้  **(1) กรณีบุคคลธรรมดา '
                         'หรือผู้มีเงินได้ตามมาตรา 40 แห่งประมวลรัษฎากร**  1. '
                         'ผู้มีเงินได้ 60,000 บาท  2. คู่สมรส (ไม่มีเงินได้) '
                         '60,000 บาท\n'
                         '3. ผู้มีเงินได้หรือคู่สมรสต่างฝ่ายต่างมีเงินได้ '
                         'หักลดหย่อนรวมกันได้ ไม่เกิน 120,000 บาท\n'
                         '4. บุตรชอบด้วยกฎหมายและบุตรบุญธรรม คนละ 30,000 บาท '
                         'บุตรคนที่ 2 เป็นต้นไปที่เกิดตั้งแต่ปี 2561 '
                         'หักหย่อนได้คนละ 60,000 บาท\n'
                         '5. ค่าฝากครรภ์และค่าคลอดบุตร '
                         'หักลดหย่อนเท่าที่จ่ายจริง แต่ไม่เกิน 60,000 บาท '
      

In [ ]:
prompt_template = """System Role
---
You are a Thai-tax specialist from the Revenue Department.
Provide accurate, concise, and professional answers.

VARIABLE BLOCK  ― replace once, no duplicates
---
Context:
{context}

Question:
{question}

Answering Instructions
---
- Read the **Context section above** and answer the Question in clear Thai.
- Base every statement strictly on that Context.
- If the Context does not fully cover the issue, write: “ต้องค้นคว้าเพิ่มเติม”
- If you still cannot answer, output exactly: **“Please try again.”**
- If the question is not tax-related, say the service is for tax questions only.
- If the Context might be outdated, warn that regulations may have changed.
- When asked about forms, procedures, or deductions, give a concise overview.
- Use minimal jargon; include short examples when helpful.
- Do not pull in external information except widely known Thai-tax basics.
- Tone: courteous, neutral, direct, and trustworthy.


Example
---
Question (example):  “How can I deduct my taxes for tax year 2024?”
Context (example):   <latest deduction rules for 2024>
Expected Answer:
  “สำหรับปีภาษี 2567 ผู้มีเงินได้สามารถหักลดหย่อน … (อ้างอิง Context)
   อย่างไรก็ตาม ควรตรวจสอบประกาศล่าสุดของกรมสรรพากรก่อนยื่นแบบ”

Fallback (no valid answer)
---
If none of the rules above yield an answer, output one line only:
**“Please try again.”**
"""


In [82]:
import requests
import json

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
model_name = "deepseek/deepseek-r1-distill-qwen-32b:free"

def find_context_for_input(question, top_k=5):
    response = find_similar_questions(question, top_k=top_k)
    matched_len = len(response["matches"])
    print(f"Found {matched_len} matches")
    return response


def send_query_req_to_openrouter(
    question, prompt_template, OPENROUTER_API_KEY=OPENROUTER_API_KEY, model_name=model_name
):
    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": "Bearer " + OPENROUTER_API_KEY,
            "Content-Type": "application/json",
        },
        data=json.dumps(
            {
                "model": model_name,
                "messages": [
                    {"role": "system", "content": prompt_template},
                    {"role": "user", "content": question},
                ],
            }
        ),
    )

    return response


In [ ]:
def map_response_to_context_prompt(response):
    """
    Map query response from pinecone to the string and this string will put inject to the prompt
    """
    result = []
    for match in response["matches"]:  # id, metadata{answer, question}, score, values
        answer = match["metadata"]["answer"]
        result.append(answer)

    result = "".join([f"<start_context>{context}<end_context>" for context in result])

    return result


test_question = "คนไทยเสียภาษีเท่าไร?"
question_context_response = find_context_for_input(test_question)
prepared_prompt_context = map_response_to_context_prompt(question_context_response)

response = send_query_req_to_openrouter(question, prepared_prompt_context)


Found 5 matches


In [89]:
response.json()


{'id': 'gen-1745688947-XJVMphnLDwriObr86Zvf',
 'provider': 'Nineteen',
 'model': 'deepseek/deepseek-r1-distill-qwen-32b:free',
 'object': 'chat.completion',
 'created': 1745688947,
 'choices': [{'logprobs': {'content': [], 'refusal': []},
   'finish_reason': 'stop',
   'native_finish_reason': 'stop',
   'index': 0,
   'message': {'role': 'assistant',
    'content': 'ข้อมูลภาษีเงินได้บุคคลธรรมดาสำหรับปีภาษี 2568 ไม่ได้ระบุไว้ในเอกสารที่ให้มา ข้อมูลที่มีอาจไม่ครอบคลุมหรืออาจล้าสมัย สำหรับข้อมูลล่าสุดเกี่ยวกับภาษีปี 2568 คุณควรตรวจสอบกับกรมสรรพากรหรือแหล่งข้อมูลทางการอื่น ๆ ที่ให้ข้อมูลเกี่ยวกับภาษีปี 2568 อย่างเป็นทางการ',
    'refusal': None,
    'reasoning': '\nOkay, ฉันต้องหาข้อมูลเกี่ยวกับภาษีปี 2568 ตามที่ผู้ใช้ถามมา ฉันได้ดูเอกสารที่ให้มาแล้ว แต่เอกสารส่วนใหญ่เกี่ยวข้องกับปีภาษี 2560 หรือก่อนหน้านั้น ไม่มีข้อมูลเกี่ยวกับปี 2568 อย่างชัดเจน\n\nฉันควรตรวจสอบแหล่งข้อมูลอื่น ๆ หรืออาจต้องบอกผู้ใช้ให้ตรวจสอบกับกรมสรรพากรเพื่อความแม่นยำ หรืออาจมีการปรับเปลี่ยนในปี 2568 ที่ฉันไม่ทราบ ฉันไ

In [90]:
response.json()["choices"][0]["message"]["content"]


'ข้อมูลภาษีเงินได้บุคคลธรรมดาสำหรับปีภาษี 2568 ไม่ได้ระบุไว้ในเอกสารที่ให้มา ข้อมูลที่มีอาจไม่ครอบคลุมหรืออาจล้าสมัย สำหรับข้อมูลล่าสุดเกี่ยวกับภาษีปี 2568 คุณควรตรวจสอบกับกรมสรรพากรหรือแหล่งข้อมูลทางการอื่น ๆ ที่ให้ข้อมูลเกี่ยวกับภาษีปี 2568 อย่างเป็นทางการ'